In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

Empresas Programas por empresa y empresas desde account


In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
    ]
]
Empresas_SF.shape


(7302, 10)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(7005, 5)

In [4]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(7308, 15)

In [5]:
Programas_pacientes = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
        "Vigencia__c": "Vigente",
        "Corredor__c": "Corredor",
        "Pa_s_del_Paciente__c": "Pais",
    }
)
Programas_pacientes = Programas_pacientes[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]


fin empresas y comienzo de rescate de pacientes


In [6]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape


(1764461, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [7]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1358215, 4)

Union de pacientes programas con las empresas


In [8]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_pacientes,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1358215, 15)

In [9]:
empresas_unidas = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)
empresas_unidas.shape


(1358215, 24)

In [10]:
empresas_unidas = empresas_unidas.rename(columns={"Id": "Id Pacientes Programas"})
empresas_unidas = empresas_unidas[
    [
        "Id Pacientes Programas",
        "Paciente__c",
        "Programas_por_Empresas__c",
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Id Account Paciente",
        "Rut",
        "Nombre",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Sexo",
        "Telefono",
        "Tipo de Documento",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]

empresas_unidas.shape


(1358215, 23)

In [11]:
# cantidad_de_PAcientes = pd.DataFrame()
def cantidad_gente(corredor):
    empresas_unidas2 = empresas_unidas[empresas_unidas["Corredor"] == corredor]
    empresas_unidas2 = empresas_unidas2.drop_duplicates(
        subset=["Rut", "Empleador", "Poliza", "Nombre Empresa"], keep="last"
    )
    cantidad_de_PAcientes = empresas_unidas2.groupby(["Poliza", "Empleador"])[
        "Paciente__c"
    ].count()
    cantidad_de_PAcientes = cantidad_de_PAcientes.reset_index()
    # cantidad_de_PAcientes = cantidad_de_PAcientes[cantidad_de_PAcientes["Poliza","Empleador","count"]]
    Archivo_gente = "F:/Reportes SF/MetLife/usuarios/" + corredor + "_gente.xlsx"
    cantidad_de_PAcientes.to_excel(Archivo_gente, index=False)
    cantidad_de_PAcientes.shape

Terminado la visualizacion de las empresas y pacientes que se muestran en los reportes


Descarga desde Citas de Servicios


In [12]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio["Empresas"] = np.nan
# Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name'])
Citas_de_servicio["Empresas Id"] = np.nan
np.matrix
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",
        "RUT__c",
        "Empresas",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "Empresas Id",
        "AccountId",
        "Clinica__c",
    ]
]
Citas_de_servicio["AppointmentNumber"] = "F" + Citas_de_servicio["AppointmentNumber"]

Citas_de_servicio.head()

,AppointmentNumber,RUT__c,Empresas,CreatedDate,Tipo_de_la_cita__c,Empresas Id,AccountId,Clinica__c
0,FSA-16763,128060006,NaN,2023-04-20T23:17:29.000+0000,Video,NaN,0018c00002gl5fhAAA,Nutrición
1,FSA-17166,186247981,NaN,2023-04-24T22:22:01.000+0000,Video,NaN,0018c00002iwWAuAAM,Nutrición
2,FSA-17173,186318072,NaN,2023-04-24T22:56:10.000+0000,Phone,NaN,0018c00002azrebAAA,Nutrición
3,FSA-17196,97073430,NaN,2023-04-25T14:21:40.000+0000,Phone,NaN,0018c00002amLcMAAU,Nutrición
4,FSA-16876,183801422,NaN,2023-04-21T19:42:01.000+0000,Phone,NaN,0018c00002amLvkAAE,Nutrición


In [13]:
Citas_de_servicio.to_excel(
    "F:/Empresas SF/reportes/citas de servicio.xlsx", index=False
)

In [14]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2["Clinica__c"] = "Orientación Médica"
Citas_de_servicio2["Tipo_de_la_cita__c"] = "Agendamiento"
Citas_de_servicio2["Empresas"] = np.nan
Citas_de_servicio2["Empresas Id"] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",
        "RUT__c",
        "Empresas",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "Empresas Id",
        "AccountId",
        "Clinica__c",
    ]
]
Citas_de_servicio2.shape

(23689, 8)

In [15]:
# unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",
        "Clinica__c": "Clínica",
    }
)

Gran_Citas_deservicio.shape

(47378, 8)

In [16]:
# Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_deporte["clinica"] = "Deporte"
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_deporte.shape

(3607, 8)

In [17]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)


ficha_nutricion.shape

(8347, 8)

In [18]:
# Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Apssex["clinica"] = "Asesoría Psicológica en Salud Sexual"
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)

ficha_Apssex.shape

(31, 8)

In [19]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia.Motivo_de_consulta_manifiesto__c.fillna(0, inplace=True)
ficha_Psicologia["Motivo_de_consulta_manifiesto__c"] = np.where(
    ficha_Psicologia["Motivo_de_consulta_manifiesto__c"] == 0,
    "Otras Derivaciones",
    ficha_Psicologia["Motivo_de_consulta_manifiesto__c"],
)
# ficha_Psicologia['77'] = len(ficha_Psicologia['Motivo_de_consulta_manifiesto__c'])
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
    }
)

ficha_Psicologia.shape

(5802, 9)

In [20]:
# Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno["clinica"] = "Clínica Del Sueño"
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Sueno.shape

(758, 8)

In [21]:
# Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico["clinica"] = "Pacientes Crónicos"
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Cronico.shape

(2487, 8)

In [22]:
# Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Telemedicina["clinica"] = "Telemedicina"
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Telemedicina.shape

(11698, 8)

In [23]:
# Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Veterinaria["clinica"] = "Veterinaria"
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Veterinaria.shape

(80, 8)

In [24]:
# Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Orientacion["clinica"] = "Orientación Médica"
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Orientacion.shape


(217, 8)

In [25]:
# Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Front["clinica"] = "Orientación Médica"
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Front.shape


(3424, 8)

In [26]:
# Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,   Programa__c   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Programa__c",
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "Programa__c": "Clínica",
    }
)
ficha_general.shape

(3605, 8)

In [27]:
# Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Ingreso_dep["clinica"] = "Ingreso Deporte Seguro"
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
    }
)
ficha_Ingreso_dep.shape

(268, 8)

In [28]:
Clinicas = pd.concat(
    [
        ficha_Front,
        ficha_Ingreso_dep,
        ficha_Orientacion,
        ficha_Veterinaria,
        ficha_Telemedicina,
        ficha_Cronico,
        ficha_Sueno,
        ficha_Psicologia,
        ficha_Apssex,
        ficha_nutricion,
        ficha_deporte,
        ficha_general,
    ],
    axis=0,
)
Clinicas.shape

(40324, 9)

In [29]:
Reortes_general = pd.concat([Clinicas, Gran_Citas_deservicio], axis=0)
Reortes_general.shape

(87702, 9)

In [30]:
Reortes_general["Dia"] = pd.to_datetime(Reortes_general["Fecha"]).dt.day

Reortes_general["Mes"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month

Reortes_general["semana"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month_name()

Reortes_general["Año"] = pd.to_datetime(Reortes_general["Fecha"]).dt.year


# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime(
    "%d-%m-%Y"
)
general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
}


clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    "Seguimiento de Pacientes Crónicos": "Pacientes Crónicos",
    "Orientación Médica": "Orientación Médica",
    "Asesoría Deportiva": "Deporte",
    "Asistencia Emocional": "Psicología",
    "Asistencia Nutricional": "Nutrición",
    "Clínica APSSex": "Asesoría Psicológica en Salud Sexual",
    "Clínica Del Sueño": "Clínica Del Sueño",
    "Clínica Veterinaria": "Veterinaria",
    "Ingreso Deportivo Seguro": "Deporte",
    "Clínica Mis Primeros Pasos": "Mis Primeros Pasos",
    "Ingreso Deporte Seguro": "Deporte",
    "APSSex": "Asesoría Psicológica en Salud Sexual",
    "Médico Cirujano": "Telemedicina",
    "Kinesióloga": "Deporte",
    "Coordinadora General de Operaciones": "Nutrición",
    "Coordinadora de Equipo de Nutrición y Deporte": "Deporte",
    "Encargado de Equipo Psicología": "Psicología",
    "Psicólogo": "Psicología",
    "Coordinadora de Telemedicina, Enfermería, Maternidad, Dental y Veterinaria": "Telemedicina",
}
Reortes_general.replace({"Clínica": clinicas_}, inplace=True)

Clinicas_metlife = {
    "Mis Primeros Pasos": "Pacientes Crónicos",
    "Ingreso Deporte Seguro": "Deporte",
    "Veterinaria": "Orientación Médica",
}

Canal = {
    "Phone": "Teléfono",
    "Video": "Videollamada",
    "Entrante - Chat": "Entrante - Chat Online",
    "Videollamda": "Videollamada",
    "Teléfono": "LLamada",
}
Reortes_general.replace({"Canal": Canal}, inplace=True)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}
Reortes_general["Meses"] = Reortes_general["Mes"]
Reortes_general.replace({"Meses": meses}, inplace=True)
Reortes_general["Fecha_chilena"] = Reortes_general["Fecha_chilena"].dt.tz_localize(None)
Reortes_general.Rut.fillna("0", inplace=True)
Reortes_general.Canal.fillna("Sin Canal", inplace=True)

In [31]:
Reortes_general.head()


,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Motivo de consulta,Dia,Mes,semana,Año,Fecha_chilena,Fecha_chilena2,Meses
0,FFF-0002,MOAI100308MJCRCSA4,None,2022-12-08T17:20:22.000+0000,Sin Canal,None,0018c00002d7jlWAAQ,Orientación Médica,NaN,8,12,December,2022,2022-12-08 17:20:22,08-12-2022,Diciembre
1,FFF-0003,AUGI770101MJCNNR06,Ambar MX,2022-12-08T21:13:52.000+0000,Saliente - Llamada,a1Y8c00000C6THvEAN,0018c00002cXy5OAAS,Orientación Médica,NaN,8,12,December,2022,2022-12-08 21:13:52,08-12-2022,Diciembre
2,FFF-0005,HESJ910818MVZRNN01,ICSI MX,2022-12-14T20:23:06.000+0000,Sin Canal,a1Y8c00000C6TKKEA3,0018c00002cXzcdAAC,Orientación Médica,NaN,14,12,December,2022,2022-12-14 20:23:06,14-12-2022,Diciembre
3,FFF-0006,RARA950705HNLMSN07,Ambar MX,2023-01-02T21:49:13.000+0000,Sin Canal,a1Y8c00000C6THlEAN,0018c00002cXy5EAAS,Orientación Médica,NaN,2,1,January,2023,2023-01-02 21:49:13,02-01-2023,Enero
4,FFF-0007,185410072,Colaboradores Care Assistance Chile SPA,2023-01-20T16:06:57.000+0000,Entrante - Llamada,a1Y8c00000C6UcYEAV,0018c00002amV70AAE,Orientación Médica,NaN,20,1,January,2023,2023-01-20 16:06:57,20-01-2023,Enero


In [32]:
empresas_unidas.head()


,Id Pacientes Programas,Paciente__c,Programas_por_Empresas__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Id Account Paciente,...,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A,Vigente,Corredor,Pais
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,0018c00002amLbpAAE,...,None,None,Mujer,967786319,RUT,EMP6008,A-0173,SI,Directo BCH,a1U8c000005dAtnEAE
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbqAAE,...,None,None,Mujer,966906983,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbrAAE,...,None,None,Mujer,988697805,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbsAAE,...,None,None,Hombre,953757757,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLc2AAE,...,None,None,Mujer,982058845,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE


In [33]:
Reportes_con_empresas = pd.merge(
    left=Reortes_general,
    right=empresas_unidas,
    how="left",
    left_on="Id Paciente",
    right_on="Id Account Paciente",
)


Reportes_con_empresas.shape


(105417, 39)

In [34]:
Reportes_con_empresas["Rut_x"].str.upper()
Reportes_con_empresas["Rut_y"].str.upper()

Reportes_con_empresas.Empleador_x.fillna(
    Reportes_con_empresas["Nombre Empresa"], inplace=True
)

In [35]:
Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)


Salida de datos


In [36]:
reportes = Reportes_con_empresas[
    [
        "Codigo Ficha",
        "Rut_x",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador_x",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]


reportes.drop(reportes[(reportes["Rut_x"] == "153298939")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "191710940")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "182470082")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "16277865")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "HEGJ930928HMNRRL06")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "1122345")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "28131394")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "34005945")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "255356798")].index, inplace=True)




Pais_SF = {"a1U8c000005dAtnEAE": "Chile", "a1U8c000005dAtsEAE": "México"}
reportes.replace({"Pais": Pais_SF}, inplace=True)

reportes = reportes.rename(columns={"Rut_x": "Rut", "Empleador_x": "Empleador"})
reportes["Motivo de consulta"].fillna(0, inplace=True)
reportes["Motivo de consulta"] = np.where(
    (reportes["Clínica"] == "Psicología") & (reportes["Motivo de consulta"] == 0),
    "Ingreso a Psicologia",
    reportes["Motivo de consulta"],
)
reportes.Clínica.fillna("Orientación Médica", inplace=True)
reportes = reportes.drop_duplicates(
    ["Clínica", "Empleador", "Nombre Empresa", "Codigo Ficha"], keep="last"
)

reportes["Rut"] = reportes["Rut"].str.replace("-", "")
reportes.replace({"Sexo": general_Sexo}, inplace=True)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1707868384.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reportes.drop(reportes[(reportes["Rut_x"] == "153298939")].index, inplace=True)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1707868384.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reportes.drop(reportes[(reportes["Rut_x"] == "191710940")].index, inplace=True)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1707868384.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [37]:
reportes.shape

(99048, 27)

In [38]:
""" Reportes_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes_completos.xlsx", index=False
) """


Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)
reportes.to_excel("C:/Users/jmartinez/Documents/reportes.xlsx", index=False)

In [39]:
general_todo = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"])]
general_todo.to_excel("F:/Reportes SF/General/General_Base.xlsx", index=False)


In [40]:
reportes = reportes[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica", 
        "Motivo de consulta",       
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]
Bhp2 = reportes[
    (reportes["Clínica"] == "Psicología") & (reportes["Holding"] == "Grupo BHP")
]
Bhp2 = Bhp2[["Clínica", "Motivo de consulta", "Fecha_chilena"]]
Bhp2.to_excel("F:/Reportes SF/Vida Security/BHP_Base2.xlsx", index=False)

reportes["Motivo de consulta"] = ""




Banco Estado


In [41]:
Banco_Estado = reportes[
    (reportes["Corredor"] == "Bancoestado Corredores De Seguros S.A.")
    & (reportes["Holding"] != "Sin Holding Asociado")
]
Banco_Estado = Banco_Estado[
    (Banco_Estado["Empleador"] == Banco_Estado["Nombre Empresa"])
]

Banco_Estado.to_excel(
    "F:/Reportes SF/Banco Estado Metlife/Banco_Estado_Base.xlsx", index=False
)

ZURICH


In [42]:
Zurich = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Zurich.to_excel("F:/Reportes SF/Zurich/Zurich_Base.xlsx", index=False)

Amsa = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Antofagasta Minerals S.A.")
]
Amsa.to_excel("F:/Reportes SF/Zurich/Amsa.xlsx", index=False)

Bechtel = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Bechtel")
]
Bechtel.to_excel("F:/Reportes SF/Zurich/Bechtel.xlsx", index=False)


VIDA SECURITY


In [43]:
Vida_security = reportes[
    (reportes["Campaña"] == "Vida Security")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Vida_security = Vida_security[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Meses",
        "Año",
        "Canal",
        "Clínica",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]
Vida_security.to_excel(
    "F:/Reportes SF/Vida Security/Vida_Security_Base.xlsx", index=False
)


Bhp = reportes[
    (reportes["Holding"] == "Grupo BHP")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Bhp = Bhp[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Meses",
        "Año",
        "Canal",
        "Clínica",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]



Bhp.to_excel("F:/Reportes SF/Vida Security/BHP_Base.xlsx", index=False)



Sqm = reportes[
    (reportes["Holding"] == "Grupo S.Q.M.")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Sqm.to_excel("F:/Reportes SF/Vida Security/SQM_Base.xlsx", index=False)


Vs_mercer = reportes[
    (reportes["Corredor"] == "Mercer") & (reportes["Campaña"] == "Vida Security")
]
Vs_mercer = Vs_mercer[(Vs_mercer["Empleador"] == Vs_mercer["Nombre Empresa"])]
Vs_mercer.to_excel("F:/Reportes SF/Vida Security/Vs_mercer_Base.xlsx", index=False)

Ferronor = reportes[
    (reportes["Holding"] == "Ferronor") & (reportes["Campaña"] == "Vida Security")
]
Ferronor = Ferronor[(Ferronor["Empleador"] == Ferronor["Nombre Empresa"])]
Ferronor.to_excel("F:/Reportes SF/Vida Security/Ferronor_Base.xlsx", index=False)

Itau = reportes[
    (reportes["Holding"] == "Itaú") & (reportes["Campaña"] == "Vida Security")
]
Itau = Itau[(Itau["Empleador"] == Itau["Nombre Empresa"])]
Itau.to_excel("F:/Reportes SF/Vida Security/Itau_Base.xlsx", index=False)

Enami = Vida_security[
    (reportes["Holding"] == "Enami") & (reportes["Campaña"] == "Vida Security")
]
Enami = Enami[(Enami["Empleador"] == Enami["Nombre Empresa"])]
Enami.to_excel("F:/Reportes SF/Vida Security/Enami_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1398605690.py:106: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Enami = Vida_security[


MERCER

In [44]:
Mercer = reportes[
    (reportes["Campaña"] == "Mercer")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Mercer.to_excel("F:/Reportes SF/Mercer/Mercer_Base.xlsx", index=False)

SURA


In [45]:
Sura = reportes[
    (reportes["Campaña"] == "Sura")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Sura.to_excel("F:/Reportes SF/Sura/Sura_Base.xlsx", index=False)

Pesco


In [46]:
Pesco = reportes[
    (reportes["Campaña"] == "Pesco")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Pesco.to_excel("F:/Reportes SF/Pesco/Pesco_Base.xlsx", index=False)

Milan Fajanovic


In [47]:
Milan = reportes[
    (reportes["Campaña"] == "Milan Fajanovic")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Milan.to_excel("F:/Reportes SF/Milan Fajanovic/Milan_Fajanovic_Base.xlsx", index=False)

Wom


In [48]:
wom = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Empleador"] == "Wom S.A.")
]
wom.to_excel("F:/Reportes SF/Wom/Wom_Base.xlsx", index=False)


In [49]:
Forus = reportes[
    (reportes["Campaña"] == "Forus")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Forus.to_excel("F:/Reportes SF/Forus/Forus_Base.xlsx", index=False)

In [50]:
Cruz = reportes[
    (reportes["Campaña"] == "Farmacias Cruz Verde")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Cruz.to_excel("F:/Reportes SF/Cruz Verde/Cruz_Base.xlsx", index=False)
Cruz.shape

(5140, 27)

METLIFE


In [51]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]


Metlife.replace({"Clínica": Clinicas_metlife}, inplace=True)

Metlife.to_excel("F:/Reportes SF/MetLife/Metlife_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\3111412387.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Metlife.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [52]:
Conosur = reportes[
    (reportes["Codigo EMP"] == "EMP5976")
    | (reportes["Codigo EMP"] == "EMP5977")
    | (reportes["Codigo EMP"] == "EMP5978")
    | (reportes["Codigo EMP"] == "EMP5979")
    | (reportes["Codigo EMP"] == "EMP5980")
    | (reportes["Codigo EMP"] == "EMP5981")
    | (reportes["Codigo EMP"] == "EMP5982")
    | (reportes["Codigo EMP"] == "EMP5983")
    | (reportes["Codigo EMP"] == "EMP5984")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP7431")
    | (reportes["Codigo EMP"] == "EMP2828")
    | (reportes["Corredor"] == "Cono Sur Corredores De Seguros Ltda.")
]
Conosur.replace({"Clínica": Clinicas_metlife}, inplace=True)
Conosur = Conosur[(Conosur["Empleador"] == Conosur["Nombre Empresa"])]
Conosur.to_excel("F:/Reportes SF/MetLife/Conosur_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2896393119.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Conosur.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [53]:
Burgos = reportes[
    (reportes["Codigo EMP"] == "EMP5924")
    | (reportes["Codigo EMP"] == "EMP5925")
    | (reportes["Codigo EMP"] == "EMP5926")
    | (reportes["Codigo EMP"] == "EMP5927")
    | (reportes["Codigo EMP"] == "EMP5928")
    | (reportes["Codigo EMP"] == "EMP5971")
    | (reportes["Codigo EMP"] == "EMP5972")
    | (reportes["Codigo EMP"] == "EMP5973")
    | (reportes["Codigo EMP"] == "EMP5974")
    | (reportes["Codigo EMP"] == "EMP5975")
    | (reportes["Codigo EMP"] == "EMP7259")
    | (reportes["Codigo EMP"] == "EMP6536")
    | (reportes["Codigo EMP"] == "EMP6959")
    | (reportes["Codigo EMP"] == "EMP6388")
    | (reportes["Corredor"] == "Burgos Y Cía. Corredores De Seguros Ltda.")
]

Burgos = Burgos[(Burgos["Empleador"] == Burgos["Nombre Empresa"])]
Burgos.replace({"Clínica": Clinicas_metlife}, inplace=True)
Burgos.to_excel("F:/Reportes SF/MetLife/Burgos_Base.xlsx", index=False)

In [54]:
Ultramar = reportes[
    (reportes["Codigo EMP"] == "EMP9232")
    | (reportes["Codigo EMP"] == "EMP9233")
    | (reportes["Codigo EMP"] == "EMP9234")
    | (reportes["Codigo EMP"] == "EMP9235")
    | (reportes["Codigo EMP"] == "EMP9236")
    | (reportes["Codigo EMP"] == "EMP9237")
    | (reportes["Codigo EMP"] == "EMP9238")
    | (reportes["Codigo EMP"] == "EMP9239")
    | (reportes["Codigo EMP"] == "EMP9240")
    | (reportes["Codigo EMP"] == "EMP5955")
    | (reportes["Codigo EMP"] == "EMP5958")
    | (reportes["Codigo EMP"] == "EMP5968")
    | (reportes["Codigo EMP"] == "EMP6713")
    | (reportes["Codigo EMP"] == "EMP7066")
    | (reportes["Codigo EMP"] == "EMP7625")
]

Ultramar = Ultramar[(Ultramar["Empleador"] == Ultramar["Nombre Empresa"])]
Ultramar.replace({"Clínica": Clinicas_metlife}, inplace=True)
Ultramar.to_excel("F:/Reportes SF/MetLife/Ultramar_Base.xlsx", index=False)

In [55]:
Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_sk = pd.read_excel(Antiguo)

reportes2 = pd.concat([reportes, Antiguo_sk])


sK = reportes2[
    (reportes2["Poliza"] == "340010027")
    | (reportes2["Poliza"] == "340010109")
    | (reportes2["Poliza"] == "340013106")
    | (reportes2["Poliza"] == "340013107")
    | (reportes2["Poliza"] == "340013108")
    | (reportes2["Poliza"] == "340013109")
    | (reportes2["Poliza"] == "340013110")
    | (reportes2["Poliza"] == "340013111")
    | (reportes2["Poliza"] == "340013112")
    | (reportes2["Poliza"] == "340013113")
    | (reportes2["Poliza"] == "340013114")
    | (reportes2["Poliza"] == "340013115")
    | (reportes2["Poliza"] == "340013116")
    | (reportes2["Poliza"] == "340013118")
    | (reportes2["Poliza"] == "340013120")
    | (reportes2["Poliza"] == "340013127")
    | (reportes2["Poliza"] == "340013128")
    | (reportes2["Poliza"] == "340013129")
    | (reportes2["Poliza"] == "340013130")
    | (reportes2["Poliza"] == "340013131")
    | (reportes2["Poliza"] == "340013132")
    | (reportes2["Poliza"] == "340013218")
    | (reportes2["Poliza"] == "340013873")
    | (reportes2["Poliza"] == "340014021")
    | (reportes2["Poliza"] == "340016184")
    | (reportes2["Poliza"] == "340016185")
    | (reportes2["Poliza"] == "340018745")
    | (reportes2["Poliza"] == "340018746")
    | (reportes2["Poliza"] == "340018747")
    | (reportes2["Poliza"] == "340019079")
    | (reportes2["Poliza"] == "340020289")
    | (reportes2["Poliza"] == "340020315")
    | (reportes2["Poliza"] == "340022643")
    | (reportes2["Poliza"] == "340022652")
    | (reportes2["Poliza"] == "340022662")
    | (reportes2["Poliza"] == "340022664")
    | (reportes2["Poliza"] == "340022665")
    | (reportes2["Poliza"] == "340022668")
    | (reportes2["Poliza"] == "340023297")
    | (reportes2["Poliza"] == "340023397")
    
]

sK.Clínica.fillna("Orientación Médica", inplace=True)
sK.replace({"Sexo": general_Sexo}, inplace=True)
sK.replace({"Clínica": Clinicas_metlife}, inplace=True)
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}
Empresas_Benefits = {
    "BancoChile": "Banco De Chile",
    "bancochile": "Banco De Chile",
    "Fundacion De Salud El Teniente": "Fundación De Salud El Teniente",
    "Serv Biene. Fundación Arturo López Pérez": "Serv. Bienestar Fundación Arturo López Pérez",
}

sK.replace({"Nombre Empresa": Empresas_Benefits}, inplace=True)
sK.replace({"Empleador": Empresas_Benefits}, inplace=True)
sK.replace({"Clínica": Lorena_clinicas}, inplace=True)

sK = sK[(sK["Empleador"] == sK["Nombre Empresa"])]
sK.replace({"Clínica": Clinicas_metlife}, inplace=True)
sK.to_excel("F:/Reportes SF/MetLife/sK_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1073128510.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sK.Clínica.fillna("Orientación Médica", inplace=True)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1073128510.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sK.replace({"Sexo": general_Sexo}, inplace=True)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1073128510.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

EY


In [56]:
Ey = reportes[
    (reportes["Campaña"] == "EY")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Ey["Nombre"] = "NoN"
Ey["Motivo de consulta"] = "NoN"

Ey.to_excel("F:/Reportes SF/EY/EY_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1813628391.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ey["Nombre"] = "NoN"
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1813628391.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ey["Motivo de consulta"] = "NoN"


In [57]:
Thb_Seguros = reportes[
    (reportes["Campaña"] == "THB Seguros COVID")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Thb_Seguros.to_excel(
    "F:/Reportes SF/THB Seguros Covid/Thb_Seguros_Base.xlsx", index=False
)

""" ( reportes["Codigo EMP"] == "EMP5924")
| (reportes["Codigo EMP"] == "EMP5925")
| (reportes["Codigo EMP"] == "EMP5926")
| (reportes["Codigo EMP"] == "EMP5927")
| (reportes["Codigo EMP"] == "EMP5928")
| (reportes["Codigo EMP"] == "EMP5971")
| (reportes["Codigo EMP"] == "EMP5972")
| (reportes["Codigo EMP"] == "EMP5973")
| (reportes["Codigo EMP"] == "EMP5974")
| (reportes["Codigo EMP"] == "EMP5975")
| (reportes["Codigo EMP"] == "EMP7259")
| (reportes["Codigo EMP"] == "EMP6536")
| (reportes["Codigo EMP"] == "EMP6959")
| (reportes["Codigo EMP"] == "EMP6388")

    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP5681")
    | (reportes["Codigo EMP"] == "EMP5682")
    | (reportes["Codigo EMP"] == "EMP5684")
    | (reportes["Codigo EMP"] == "EMP6274")
    | (reportes["Codigo EMP"] == "EMP5685")
    | (reportes["Codigo EMP"] == "EMP5686")
    | (reportes["Codigo EMP"] == "EMP5687")
    | (reportes["Codigo EMP"] == "EMP5688")
    | (reportes["Codigo EMP"] == "EMP5689")
    | (reportes["Codigo EMP"] == "EMP5690")
    | (reportes["Codigo EMP"] == "EMP5691")

    | (reportes["Codigo EMP"] == "EMP5692")
    | (reportes["Codigo EMP"] == "EMP5693")
    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5695")
    | (reportes["Codigo EMP"] == "EMP5696")
    | (reportes["Codigo EMP"] == "EMP5697")
    | (reportes["Codigo EMP"] == "EMP5698")
    | (reportes["Codigo EMP"] == "EMP5639")
    | (reportes["Codigo EMP"] == "EMP5640")
    | (reportes["Codigo EMP"] == "EMP5641")
    | (reportes["Codigo EMP"] == "EMP5642")
    | (reportes["Codigo EMP"] == "EMP5643")
    | (reportes["Codigo EMP"] == "EMP5644")

    | (reportes["Codigo EMP"] == "EMP5645")
    | (reportes["Codigo EMP"] == "EMP5646")
    | (reportes["Codigo EMP"] == "EMP5647")
    | (reportes["Codigo EMP"] == "EMP5648")
    | (reportes["Codigo EMP"] == "EMP5649")
    | (reportes["Codigo EMP"] == "EMP5650")
    | (reportes["Codigo EMP"] == "EMP5651")
    | (reportes["Codigo EMP"] == "EMP5652")
    | (reportes["Codigo EMP"] == "EMP5653")
    | (reportes["Codigo EMP"] == "EMP5654")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP7526")
    | (reportes["Codigo EMP"] == "EMP6760")

    | (reportes["Codigo EMP"] == "EMP7678")
    | (reportes["Codigo EMP"] == "EMP6888")
    | (reportes["Codigo EMP"] == "EMP6444")
    | (reportes["Codigo EMP"] == "EMP6406")
    | (reportes["Codigo EMP"] == "EMP5616")
    | (reportes["Codigo EMP"] == "EMP9007")
    |"""


In [58]:
Willis = reportes[(reportes["Corredor"] == "Willis Towers Watson S.A. Corred. De Seg.")]
Willis.replace({"Clínica": Clinicas_metlife}, inplace=True)
Willis = Willis[(Willis["Empleador"] == Willis["Nombre Empresa"])]
Willis.to_excel("F:/Reportes SF/MetLife/Willis_Base.xlsx", index=False)

sKf = reportes[(reportes["Poliza"] == "340018080")]
sKf.replace({"Clínica": Clinicas_metlife}, inplace=True)
sKf = sKf[(sKf["Empleador"] == sKf["Nombre Empresa"])]
sKf.to_excel("F:/Reportes SF/MetLife/Willis_sKf_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2795123212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Willis.replace({"Clínica": Clinicas_metlife}, inplace=True)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2795123212.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sKf.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [59]:
Aon = reportes[
    (reportes["Codigo EMP"] == "EMP5627")
    | (reportes["Codigo EMP"] == "EMP9181")
    | (reportes["Codigo EMP"] == "EMP9183")
    | (reportes["Codigo EMP"] == "EMP9184")
    | (reportes["Codigo EMP"] == "EMP9185")
    | (reportes["Codigo EMP"] == "EMP9186")
    | (reportes["Codigo EMP"] == "EMP9241")
    | (reportes["Codigo EMP"] == "EMP9256")
    | (reportes["Codigo EMP"] == "EMP9187")
    | (reportes["Codigo EMP"] == "EMP9188")
    | (reportes["Codigo EMP"] == "EMP9189")
    | (reportes["Codigo EMP"] == "EMP9230")
    | (reportes["Codigo EMP"] == "EMP9359")
    | (reportes["Codigo EMP"] == "EMP9365")
    | (reportes["Codigo EMP"] == "EMP9366")
    | (reportes["Codigo EMP"] == "EMP9367")
    | (reportes["Codigo EMP"] == "EMP9368")
    | (reportes["Codigo EMP"] == "EMP5890")
    | (reportes["Codigo EMP"] == "EMP5891")
    | (reportes["Codigo EMP"] == "EMP5892")
    | (reportes["Codigo EMP"] == "EMP5893")
    | (reportes["Codigo EMP"] == "EMP5894")
    | (reportes["Codigo EMP"] == "EMP5895")
    | (reportes["Codigo EMP"] == "EMP5896")
    | (reportes["Codigo EMP"] == "EMP5897")
    | (reportes["Codigo EMP"] == "EMP5898")
    | (reportes["Codigo EMP"] == "EMP5899")
    | (reportes["Codigo EMP"] == "EMP5930")
    | (reportes["Codigo EMP"] == "EMP5931")
    | (reportes["Codigo EMP"] == "EMP5932")
    | (reportes["Codigo EMP"] == "EMP5933")
    | (reportes["Codigo EMP"] == "EMP5934")
    | (reportes["Codigo EMP"] == "EMP5935")
    | (reportes["Codigo EMP"] == "EMP5936")
    | (reportes["Codigo EMP"] == "EMP5937")
    | (reportes["Codigo EMP"] == "EMP5939")
    | (reportes["Codigo EMP"] == "EMP5940")
    | (reportes["Codigo EMP"] == "EMP5941")
    | (reportes["Codigo EMP"] == "EMP5942")
    | (reportes["Codigo EMP"] == "EMP5943")
    | (reportes["Codigo EMP"] == "EMP5944")
    | (reportes["Codigo EMP"] == "EMP5945")
    | (reportes["Codigo EMP"] == "EMP5946")
    | (reportes["Codigo EMP"] == "EMP5947")
    | (reportes["Codigo EMP"] == "EMP5949")
    | (reportes["Codigo EMP"] == "EMP9436")
    | (reportes["Codigo EMP"] == "EMP9437")
    | (reportes["Codigo EMP"] == "EMP9861")
    | (reportes["Codigo EMP"] == "EMP9862")
    | (reportes["Codigo EMP"] == "EMP9864")
    | (reportes["Codigo EMP"] == "EMP9865")
    | (reportes["Codigo EMP"] == "EMP9866")
    | (reportes["Codigo EMP"] == "EMP9873")
    | (reportes["Codigo EMP"] == "EMP9874")
    | (reportes["Codigo EMP"] == "EMP9875")
    | (reportes["Codigo EMP"] == "EMP9892")
    | (reportes["Codigo EMP"] == "EMP9893")
    | (reportes["Codigo EMP"] == "EMP9899")
    | (reportes["Codigo EMP"] == "EMP9901")
    | (reportes["Codigo EMP"] == "EMP9902")
    | (reportes["Codigo EMP"] == "EMP9903")
    | (reportes["Codigo EMP"] == "EMP9904")
    | (reportes["Codigo EMP"] == "EMP9905")
    | (reportes["Codigo EMP"] == "EMP9906")
    | (reportes["Codigo EMP"] == "EMP9907")
    | (reportes["Codigo EMP"] == "EMP9908")
    | (reportes["Codigo EMP"] == "EMP9909")
    | (reportes["Codigo EMP"] == "EMP9493")
    | (reportes["Codigo EMP"] == "EMP9494")
    | (reportes["Codigo EMP"] == "EMP9518")
    | (reportes["Codigo EMP"] == "EMP9539")
    | (reportes["Codigo EMP"] == "EMP9540")
    | (reportes["Codigo EMP"] == "EMP9558")
    | (reportes["Codigo EMP"] == "EMP9741")
    | (reportes["Codigo EMP"] == "EMP9742")
    | (reportes["Codigo EMP"] == "EMP9743")
    | (reportes["Codigo EMP"] == "EMP9745")
    | (reportes["Codigo EMP"] == "EMP9747")
    | (reportes["Codigo EMP"] == "EMP9630")
    | (reportes["Codigo EMP"] == "EMP9631")
    | (reportes["Codigo EMP"] == "EMP9632")
    | (reportes["Codigo EMP"] == "EMP9638")
    | (reportes["Codigo EMP"] == "EMP9755")
    | (reportes["Codigo EMP"] == "EMP6532")
    | (reportes["Codigo EMP"] == "EMP6538")
    | (reportes["Codigo EMP"] == "EMP6555")
    | (reportes["Codigo EMP"] == "EMP6556")
    | (reportes["Codigo EMP"] == "EMP6557")
    | (reportes["Codigo EMP"] == "EMP6558")
    | (reportes["Codigo EMP"] == "EMP6560")
    | (reportes["Codigo EMP"] == "EMP6561")
    | (reportes["Codigo EMP"] == "EMP6562")
    | (reportes["Codigo EMP"] == "EMP6563")
    | (reportes["Codigo EMP"] == "EMP6671")
    | (reportes["Codigo EMP"] == "EMP6717")
    | (reportes["Codigo EMP"] == "EMP6720")
    | (reportes["Codigo EMP"] == "EMP6786")
    | (reportes["Codigo EMP"] == "EMP6864")
    | (reportes["Codigo EMP"] == "EMP6865")
    | (reportes["Codigo EMP"] == "EMP6866")
    | (reportes["Codigo EMP"] == "EMP6867")
    | (reportes["Codigo EMP"] == "EMP6868")
    | (reportes["Codigo EMP"] == "EMP6938")
    | (reportes["Codigo EMP"] == "EMP7003")
    | (reportes["Codigo EMP"] == "EMP7004")
    | (reportes["Codigo EMP"] == "EMP7083")
    | (reportes["Codigo EMP"] == "EMP7176")
    | (reportes["Codigo EMP"] == "EMP7177")
    | (reportes["Codigo EMP"] == "EMP7269")
    | (reportes["Codigo EMP"] == "EMP7337")
    | (reportes["Codigo EMP"] == "EMP7396")
    | (reportes["Codigo EMP"] == "EMP7418")
    | (reportes["Codigo EMP"] == "EMP7513")
    | (reportes["Codigo EMP"] == "EMP7646")
    | (reportes["Codigo EMP"] == "EMP7808")
    | (reportes["Codigo EMP"] == "EMP7863")
    | (reportes["Codigo EMP"] == "EMP9183")
    | (reportes["Codigo EMP"] == "EMP9184")
    | (reportes["Codigo EMP"] == "EMP9185")
    | (reportes["Codigo EMP"] == "EMP9186")
    | (reportes["Codigo EMP"] == "EMP9256")
    | (reportes["Codigo EMP"] == "EMP9187")
    | (reportes["Codigo EMP"] == "EMP9188")
    | (reportes["Codigo EMP"] == "EMP9189")
    | (reportes["Codigo EMP"] == "EMP5890")
    | (reportes["Codigo EMP"] == "EMP5891")
    | (reportes["Codigo EMP"] == "EMP5892")
    | (reportes["Codigo EMP"] == "EMP5893")
    | (reportes["Codigo EMP"] == "EMP5894")
    | (reportes["Codigo EMP"] == "EMP5895")
    | (reportes["Codigo EMP"] == "EMP5896")
    | (reportes["Codigo EMP"] == "EMP5897")
    | (reportes["Codigo EMP"] == "EMP5898")
    | (reportes["Codigo EMP"] == "EMP5899")
    | (reportes["Codigo EMP"] == "EMP5930")
    | (reportes["Codigo EMP"] == "EMP5931")
    | (reportes["Codigo EMP"] == "EMP5932")
    | (reportes["Codigo EMP"] == "EMP5933")
    | (reportes["Codigo EMP"] == "EMP5934")
    | (reportes["Codigo EMP"] == "EMP5935")
    | (reportes["Codigo EMP"] == "EMP5936")
    | (reportes["Codigo EMP"] == "EMP5937")
    | (reportes["Codigo EMP"] == "EMP5939")
    | (reportes["Codigo EMP"] == "EMP5940")
    | (reportes["Codigo EMP"] == "EMP5941")
    | (reportes["Codigo EMP"] == "EMP5944")
    | (reportes["Codigo EMP"] == "EMP5947")
    | (reportes["Codigo EMP"] == "EMP5949")
    | (reportes["Codigo EMP"] == "EMP9436")
    | (reportes["Codigo EMP"] == "EMP9863")
    | (reportes["Codigo EMP"] == "EMP9864")
    | (reportes["Codigo EMP"] == "EMP9865")
    | (reportes["Codigo EMP"] == "EMP9866")
    | (reportes["Codigo EMP"] == "EMP9901")
    | (reportes["Codigo EMP"] == "EMP9902")
    | (reportes["Codigo EMP"] == "EMP9903")
    | (reportes["Codigo EMP"] == "EMP9904")
    | (reportes["Codigo EMP"] == "EMP9905")
    | (reportes["Codigo EMP"] == "EMP9906")
    | (reportes["Codigo EMP"] == "EMP9907")
    | (reportes["Codigo EMP"] == "EMP9908")
    | (reportes["Codigo EMP"] == "EMP9909")
    | (reportes["Codigo EMP"] == "EMP9493")
    | (reportes["Codigo EMP"] == "EMP9494")
    | (reportes["Codigo EMP"] == "EMP9539")
    | (reportes["Codigo EMP"] == "EMP9540")
    | (reportes["Codigo EMP"] == "EMP9558")
    | (reportes["Codigo EMP"] == "EMP9741")
    | (reportes["Codigo EMP"] == "EMP9742")
    | (reportes["Codigo EMP"] == "EMP9627")
    | (reportes["Codigo EMP"] == "EMP9630")
    | (reportes["Codigo EMP"] == "EMP9631")
    | (reportes["Codigo EMP"] == "EMP9632")
    | (reportes["Codigo EMP"] == "EMP9638")
    | (reportes["Codigo EMP"] == "EMP6786")
    | (reportes["Codigo EMP"] == "EMP7597")
    | (reportes["Corredor"] == "Aon Risk Services (Chile) Corred. De Seg.")
]
Aon.replace({"Clínica": Clinicas_metlife}, inplace=True)
Aon = Aon[(Aon["Empleador"] == Aon["Nombre Empresa"])]
Aon.to_excel("F:/Reportes SF/MetLife/Aon_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\861990135.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Aon.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [60]:
Aramark = reportes[
    (reportes["Codigo EMP"] == "EMP9244")
    | (reportes["Codigo EMP"] == "EMP9245")
    | (reportes["Codigo EMP"] == "EMP9246")
    | (reportes["Codigo EMP"] == "EMP9247")
    | (reportes["Codigo EMP"] == "EMP9248")
    | (reportes["Codigo EMP"] == "EMP9249")
    | (reportes["Codigo EMP"] == "EMP9250")
    | (reportes["Codigo EMP"] == "EMP9148")
    | (reportes["Codigo EMP"] == "EMP9149")
    | (reportes["Codigo EMP"] == "EMP9150")
    | (reportes["Codigo EMP"] == "EMP9151")
    | (reportes["Codigo EMP"] == "EMP9152")
    | (reportes["Codigo EMP"] == "EMP9153")
    | (reportes["Codigo EMP"] == "EMP9154")
    | (reportes["Codigo EMP"] == "EMP9155")
    | (reportes["Codigo EMP"] == "EMP9156")
    | (reportes["Codigo EMP"] == "EMP9157")
    | (reportes["Codigo EMP"] == "EMP9158")
    | (reportes["Codigo EMP"] == "EMP9159")
    | (reportes["Codigo EMP"] == "EMP9160")
    | (reportes["Codigo EMP"] == "EMP9161")
    | (reportes["Codigo EMP"] == "EMP9162")
    | (reportes["Codigo EMP"] == "EMP9163")
    | (reportes["Codigo EMP"] == "EMP9164")
    | (reportes["Codigo EMP"] == "EMP9165")
    | (reportes["Codigo EMP"] == "EMP9166")
    | (reportes["Codigo EMP"] == "EMP9167")
    | (reportes["Codigo EMP"] == "EMP5806")
    | (reportes["Codigo EMP"] == "EMP5808")
    | (reportes["Codigo EMP"] == "EMP5810")
    | (reportes["Codigo EMP"] == "EMP9266")
    | (reportes["Codigo EMP"] == "EMP9267")
    | (reportes["Codigo EMP"] == "EMP9268")
    | (reportes["Codigo EMP"] == "EMP9269")
    | (reportes["Codigo EMP"] == "EMP9270")
    | (reportes["Codigo EMP"] == "EMP9271")
    # | (reportes["Corredor"] == "Aon Risk Services (Chile) Corred. De Seg.")
]
Aramark.replace({"Clínica": Clinicas_metlife}, inplace=True)
Aramark = Aramark[(Aramark["Empleador"] == Aramark["Nombre Empresa"])]
Aramark.to_excel("F:/Reportes SF/MetLife/Aramark_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2172310933.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Aramark.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [61]:
Benefits = reportes[
    (reportes["Codigo EMP"] == "EMP9190")
    | (reportes["Codigo EMP"] == "EMP9191")
    | (reportes["Codigo EMP"] == "EMP9192")
    | (reportes["Codigo EMP"] == "EMP9193")
    | (reportes["Codigo EMP"] == "EMP9194")
    | (reportes["Codigo EMP"] == "EMP9195")
    | (reportes["Codigo EMP"] == "EMP9196")
    | (reportes["Codigo EMP"] == "EMP9197")
    | (reportes["Codigo EMP"] == "EMP9198")
    | (reportes["Codigo EMP"] == "EMP9199")
    | (reportes["Codigo EMP"] == "EMP9200")
    | (reportes["Codigo EMP"] == "EMP9201")
    | (reportes["Codigo EMP"] == "EMP9202")
    | (reportes["Codigo EMP"] == "EMP9203")
    | (reportes["Codigo EMP"] == "EMP9204")
    | (reportes["Codigo EMP"] == "EMP9205")
    | (reportes["Codigo EMP"] == "EMP9326")
    | (reportes["Codigo EMP"] == "EMP9327")
    | (reportes["Codigo EMP"] == "EMP9328")
    | (reportes["Codigo EMP"] == "EMP9340")
    | (reportes["Codigo EMP"] == "EMP9345")
    | (reportes["Codigo EMP"] == "EMP9349")
    | (reportes["Codigo EMP"] == "EMP9324")
    | (reportes["Codigo EMP"] == "EMP5908")
    | (reportes["Codigo EMP"] == "EMP5909")
    | (reportes["Codigo EMP"] == "EMP5912")
    | (reportes["Codigo EMP"] == "EMP5913")
    | (reportes["Codigo EMP"] == "EMP5914")
    | (reportes["Codigo EMP"] == "EMP5915")
    | (reportes["Codigo EMP"] == "EMP5916")
    | (reportes["Codigo EMP"] == "EMP5917")
    | (reportes["Codigo EMP"] == "EMP5919")
    | (reportes["Codigo EMP"] == "EMP5920")
    | (reportes["Codigo EMP"] == "EMP5921")
    | (reportes["Codigo EMP"] == "EMP5922")
    | (reportes["Codigo EMP"] == "EMP5923")
    | (reportes["Codigo EMP"] == "EMP6001")
    | (reportes["Codigo EMP"] == "EMP9414")
    | (reportes["Codigo EMP"] == "EMP9464")
    | (reportes["Codigo EMP"] == "EMP9840")
    | (reportes["Codigo EMP"] == "EMP9852")
    | (reportes["Codigo EMP"] == "EMP9703")
    | (reportes["Codigo EMP"] == "EMP9600")
    | (reportes["Codigo EMP"] == "EMP9601")
    | (reportes["Codigo EMP"] == "EMP9602")
    | (reportes["Codigo EMP"] == "EMP9603")
    | (reportes["Codigo EMP"] == "EMP9604")
    | (reportes["Codigo EMP"] == "EMP9605")
    | (reportes["Codigo EMP"] == "EMP9606")
    | (reportes["Codigo EMP"] == "EMP9608")
    | (reportes["Codigo EMP"] == "EMP9609")
    | (reportes["Codigo EMP"] == "EMP9610")
    | (reportes["Codigo EMP"] == "EMP9611")
    | (reportes["Codigo EMP"] == "EMP9612")
    | (reportes["Codigo EMP"] == "EMP9613")
    | (reportes["Codigo EMP"] == "EMP9614")
    | (reportes["Codigo EMP"] == "EMP9615")
    | (reportes["Codigo EMP"] == "EMP9616")
    | (reportes["Codigo EMP"] == "EMP9617")
    | (reportes["Codigo EMP"] == "EMP9618")
    | (reportes["Codigo EMP"] == "EMP9619")
    | (reportes["Codigo EMP"] == "EMP9784")
    | (reportes["Codigo EMP"] == "EMP9670")
    | (reportes["Codigo EMP"] == "EMP9674")
    | (reportes["Codigo EMP"] == "EMP9802")
    | (reportes["Codigo EMP"] == "EMP6534")
    | (reportes["Codigo EMP"] == "EMP6540")
    | (reportes["Codigo EMP"] == "EMP6564")
    | (reportes["Codigo EMP"] == "EMP6621")
    | (reportes["Codigo EMP"] == "EMP6640")
    | (reportes["Codigo EMP"] == "EMP6643")
    | (reportes["Codigo EMP"] == "EMP6650")
    | (reportes["Codigo EMP"] == "EMP6653")
    | (reportes["Codigo EMP"] == "EMP6661")
    | (reportes["Codigo EMP"] == "EMP6758")
    | (reportes["Codigo EMP"] == "EMP6763")
    | (reportes["Codigo EMP"] == "EMP6772")
    | (reportes["Codigo EMP"] == "EMP6783")
    | (reportes["Codigo EMP"] == "EMP6792")
    | (reportes["Codigo EMP"] == "EMP6829")
    | (reportes["Codigo EMP"] == "EMP6869")
    | (reportes["Codigo EMP"] == "EMP6935")
    | (reportes["Codigo EMP"] == "EMP6940")
    | (reportes["Codigo EMP"] == "EMP7005")
    | (reportes["Codigo EMP"] == "EMP7033")
    | (reportes["Codigo EMP"] == "EMP7072")
    | (reportes["Codigo EMP"] == "EMP7189")
    | (reportes["Codigo EMP"] == "EMP7243")
    | (reportes["Codigo EMP"] == "EMP7247")
    | (reportes["Codigo EMP"] == "EMP7345")
    | (reportes["Codigo EMP"] == "EMP7507")
    | (reportes["Codigo EMP"] == "EMP7508")
    | (reportes["Codigo EMP"] == "EMP7546")
    | (reportes["Codigo EMP"] == "EMP7564")
    | (reportes["Codigo EMP"] == "EMP7567")
    | (reportes["Codigo EMP"] == "EMP7617")
    | (reportes["Codigo EMP"] == "EMP7639")
    | (reportes["Codigo EMP"] == "EMP7663")
    | (reportes["Codigo EMP"] == "EMP7674")
    | (reportes["Codigo EMP"] == "EMP7702")
    | (reportes["Codigo EMP"] == "EMP7739")
    | (reportes["Codigo EMP"] == "EMP7744")
    | (reportes["Codigo EMP"] == "EMP7784")
    | (reportes["Codigo EMP"] == "EMP7785")
    | (reportes["Codigo EMP"] == "EMP9206")
    | (reportes["Codigo EMP"] == "EMP5910")
    | (reportes["Codigo EMP"] == "EMP5911")
    | (reportes["Codigo EMP"] == "EMP5918")
    | (reportes["Codigo EMP"] == "EMP9607")
    | (reportes["Corredor"] == "Benefits Corredores De Seguros SPA")
]

cantidad_gente("Benefits Corredores De Seguros SPA")


Benefits = Benefits[(Benefits["Empleador"] == Benefits["Nombre Empresa"])]

Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_Lorena = pd.read_excel(Antiguo)
Antiguo_Lorena = Antiguo_Lorena[
    (Antiguo_Lorena["Corredor"] == "Benefits Corredores De Seguros Spa")
]
Benefits = pd.concat([Benefits, Antiguo_Lorena])


Antiguo_gente = (
    "F:/Reportes SF/MetLife/usuarios/cantidad_metlife_antiguo_gente_benefits.xlsx"
)
nuevo_gente = (
    "F:/Reportes SF/MetLife/usuarios/Benefits Corredores De Seguros Spa_gente.xlsx"
)
Antiguo_gente_read = pd.read_excel(Antiguo_gente)
nuevo_gente_read = pd.read_excel(nuevo_gente)

Benefits_gente = pd.concat([Antiguo_gente_read, nuevo_gente_read])


Benefits.Clínica.fillna("Orientación Médica", inplace=True)
Benefits.replace({"Sexo": general_Sexo}, inplace=True)
Benefits.replace({"Clínica": Clinicas_metlife}, inplace=True)
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}
Empresas_Benefits = {
    "BancoChile": "Banco De Chile",
    "bancochile": "Banco De Chile",
    "Fundacion De Salud El Teniente": "Fundación De Salud El Teniente",
    "Serv Biene. Fundación Arturo López Pérez": "Serv. Bienestar Fundación Arturo López Pérez",
}
Benefits_gente.replace({"Empleador": Empresas_Benefits}, inplace=True)
Benefits.replace({"Nombre Empresa": Empresas_Benefits}, inplace=True)
Benefits.replace({"Empleador": Empresas_Benefits}, inplace=True)
Benefits.replace({"Clínica": Lorena_clinicas}, inplace=True)
Benefits = Benefits[(Benefits["Empleador"] == Benefits["Nombre Empresa"])]
Benefits.to_excel("F:/Reportes SF/MetLife/Benefits_Base.xlsx", index=False)
Benefits_gente.to_excel(
    "F:/Reportes SF/MetLife/usuarios/gente_Benefits.xlsx", index=False
)


In [62]:
wom = reportes[
    (reportes["Campaña"] == "MetLife") & (reportes["Empleador"] == "Wom S.A.")
]
wom = wom[(wom["Empleador"] == wom["Nombre Empresa"])]
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}
Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_Lorena = pd.read_excel(Antiguo)
Antiguo_Lorena = Antiguo_Lorena[(Antiguo_Lorena["Empleador"] == "Wom S.A.")]

wom = pd.concat([wom, Antiguo_Lorena])
wom.replace({"Clínica": Lorena_clinicas}, inplace=True)
wom.to_excel("F:/Reportes SF/MetLife/wom_Base.xlsx", index=False)

In [63]:
psg = reportes[
    (reportes["Campaña"] == "Grupo PSG") 
]
psg = psg[(psg["Empleador"] == psg["Nombre Empresa"])]

psg.to_excel("F:/Reportes SF/Grupo PSG/psg_Base.xlsx", index=False)

In [64]:
Interno = reportes[
    (reportes["Codigo EMP"] == "EMP5999")
    | (reportes["Codigo EMP"] == "EMP9683")
    | (reportes["Codigo EMP"] == "EMP9684")
    | (reportes["Codigo EMP"] == "EMP9685")
    | (reportes["Codigo EMP"] == "EMP9686")
    | (reportes["Codigo EMP"] == "EMP9687")
    | (reportes["Codigo EMP"] == "EMP9688")
    | (reportes["Codigo EMP"] == "EMP9689")
    | (reportes["Codigo EMP"] == "EMP9690")
    | (reportes["Codigo EMP"] == "EMP9691")
    | (reportes["Codigo EMP"] == "EMP7881")
    # | (reportes["Corredor"] == "Benefits Corredores De Seguros SPA")
]
Interno.replace({"Clínica": Clinicas_metlife}, inplace=True)
Interno = Interno[(Interno["Empleador"] == Interno["Nombre Empresa"])]
Interno.to_excel("F:/Reportes SF/MetLife/Interno_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\3001414131.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Interno.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [65]:
Lorena = reportes[
    (reportes["Codigo EMP"] == "EMP5786")
    | (reportes["Codigo EMP"] == "EMP9171")
    | (reportes["Codigo EMP"] == "EMP5785")
    | (reportes["Codigo EMP"] == "EMP5787")
    | (reportes["Codigo EMP"] == "EMP5788")
    | (reportes["Codigo EMP"] == "EMP5789")
    | (reportes["Codigo EMP"] == "EMP5790")
    | (reportes["Codigo EMP"] == "EMP5791")
    | (reportes["Codigo EMP"] == "EMP5792")
    | (reportes["Codigo EMP"] == "EMP5793")
    | (reportes["Codigo EMP"] == "EMP5794")
    | (reportes["Codigo EMP"] == "EMP9310")
    | (reportes["Codigo EMP"] == "EMP9441")
    | (reportes["Codigo EMP"] == "EMP9792")
    | (reportes["Codigo EMP"] == "EMP6830")
    | (reportes["Codigo EMP"] == "EMP6975")
    | (reportes["Codigo EMP"] == "EMP7014")
    | (reportes["Codigo EMP"] == "EMP7045")
    | (reportes["Codigo EMP"] == "EMP7060")
    | (reportes["Codigo EMP"] == "EMP7422")
    | (reportes["Corredor"] == "Lorena Contreras Ugarte")
]
cantidad_gente("Lorena Contreras Ugarte")


Lorena = Lorena[(Lorena["Empleador"] == Lorena["Nombre Empresa"])]

Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_Lorena = pd.read_excel(Antiguo)
Antiguo_Lorena = Antiguo_Lorena[
    (Antiguo_Lorena["Corredor"] == "Lorena Contreras Ugarte")
]
Lorena = pd.concat([Lorena, Antiguo_Lorena])

Antiguo_gente = "F:/Reportes SF/MetLife/usuarios/cantidad_metlife_antiguo_gente.xlsx"
nuevo_gente = "F:/Reportes SF/MetLife/usuarios/Lorena Contreras Ugarte_gente.xlsx"
Antiguo_gente_read = pd.read_excel(Antiguo_gente)
nuevo_gente_read = pd.read_excel(nuevo_gente)

Lorena_gente = pd.concat([Antiguo_gente_read, nuevo_gente_read])
Lorena_gente.to_excel("F:/Reportes SF/MetLife/usuarios/gente_Lorena.xlsx", index=False)

Lorena.Clínica.fillna("Orientación Médica", inplace=True)
Lorena.replace({"Sexo": general_Sexo}, inplace=True)
Lorena.replace({"Clínica": Clinicas_metlife}, inplace=True)
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}

Lorena.replace({"Clínica": Lorena_clinicas}, inplace=True)

Lorena.to_excel("F:/Reportes SF/MetLife/Lorena_Base.xlsx", index=False)


In [66]:
Antiguo_Lorena.head()


,Codigo Ficha,Rut,Nombre,Sexo,Ocupacion,Cargo,Departamento,Campaña,Holding,Empleador,...,Meses,Canal,Clínica,Motivo de consulta,Nombre Empresa,Codigo EMP,Codigo A,Vigente,Corredor,Pais
83,61d58cbb49839dd0a,26206235K,ACOSTA MATIE VICTORIA ALEXJA,Mujer,NaN,NaN,NaN,MetLife,NaN,Sociedad Comercial Allendes Hnos.,...,Enero,Llamada telefónica,Psicología,0,Sociedad Comercial Allendes Hnos.,0,0,SI,Lorena Contreras Ugarte,Chile
109,61d58cbb49839dd0a,26206235K,ACOSTA MATIE VICTORIA ALEXJA,Mujer,NaN,NaN,NaN,MetLife,NaN,Sociedad Comercial Allendes Hnos.,...,Enero,Llamada telefónica,Psicología,0,Sociedad Comercial Allendes Hnos.,0,0,SI,Lorena Contreras Ugarte,Chile
132,61d58cbb49839dd0a,26206235K,ACOSTA MATIE VICTORIA ALEXJA,Mujer,NaN,NaN,NaN,MetLife,NaN,Sociedad Comercial Allendes Hnos.,...,Febrero,Llamada telefónica,Psicología,0,Sociedad Comercial Allendes Hnos.,0,0,SI,Lorena Contreras Ugarte,Chile
153,61d58cb94d6959a4a,228278424,VALERO BRIONES SOFIA,Mujer,NaN,NaN,NaN,MetLife,NaN,Sociedad Comercial Allendes Hnos.,...,Febrero,Llamada telefónica,Psicología,0,Sociedad Comercial Allendes Hnos.,0,0,SI,Lorena Contreras Ugarte,Chile
170,61d58cb94d6959a4a,228278424,VALERO BRIONES SOFIA,Mujer,NaN,NaN,NaN,MetLife,NaN,Sociedad Comercial Allendes Hnos.,...,Febrero,Llamada telefónica,Psicología,0,Sociedad Comercial Allendes Hnos.,0,0,SI,Lorena Contreras Ugarte,Chile


In [67]:
Security_metlife = reportes[
    (reportes["Codigo EMP"] == "EMP5952")
    | (reportes["Codigo EMP"] == "EMP5957")
    | (reportes["Codigo EMP"] == "EMP5966")
    | (reportes["Codigo EMP"] == "EMP5989")
    | (reportes["Codigo EMP"] == "EMP6833")
    | (reportes["Codigo EMP"] == "EMP6902")
    | (reportes["Codigo EMP"] == "EMP7332")
    | (reportes["Codigo EMP"] == "EMP7661")
    | (reportes["Codigo EMP"] == "EMP9232")
    | (reportes["Codigo EMP"] == "EMP9233")
    | (reportes["Codigo EMP"] == "EMP9234")
    | (reportes["Codigo EMP"] == "EMP9235")
    | (reportes["Codigo EMP"] == "EMP9236")
    | (reportes["Codigo EMP"] == "EMP9237")
    | (reportes["Codigo EMP"] == "EMP9238")
    | (reportes["Codigo EMP"] == "EMP9239")
    | (reportes["Codigo EMP"] == "EMP9240")
    | (reportes["Codigo EMP"] == "EMP9347")
    | (reportes["Codigo EMP"] == "EMP9287")
    | (reportes["Codigo EMP"] == "EMP5950")
    | (reportes["Codigo EMP"] == "EMP5951")
    | (reportes["Codigo EMP"] == "EMP5953")
    | (reportes["Codigo EMP"] == "EMP5955")
    | (reportes["Codigo EMP"] == "EMP5956")
    | (reportes["Codigo EMP"] == "EMP5958")
    | (reportes["Codigo EMP"] == "EMP5959")
    | (reportes["Codigo EMP"] == "EMP5960")
    | (reportes["Codigo EMP"] == "EMP5961")
    | (reportes["Codigo EMP"] == "EMP5962")
    | (reportes["Codigo EMP"] == "EMP5963")
    | (reportes["Codigo EMP"] == "EMP5964")
    | (reportes["Codigo EMP"] == "EMP5965")
    | (reportes["Codigo EMP"] == "EMP5967")
    | (reportes["Codigo EMP"] == "EMP5968")
    | (reportes["Codigo EMP"] == "EMP5969")
    | (reportes["Codigo EMP"] == "EMP5990")
    | (reportes["Codigo EMP"] == "EMP5991")
    | (reportes["Codigo EMP"] == "EMP9440")
    | (reportes["Codigo EMP"] == "EMP9454")
    | (reportes["Codigo EMP"] == "EMP9455")
    | (reportes["Codigo EMP"] == "EMP9456")
    | (reportes["Codigo EMP"] == "EMP9457")
    | (reportes["Codigo EMP"] == "EMP9458")
    | (reportes["Codigo EMP"] == "EMP9459")
    | (reportes["Codigo EMP"] == "EMP9460")
    | (reportes["Codigo EMP"] == "EMP9512")
    | (reportes["Codigo EMP"] == "EMP9553")
    | (reportes["Codigo EMP"] == "EMP9669")
    | (reportes["Codigo EMP"] == "EMP6434")
    | (reportes["Codigo EMP"] == "EMP6713")
    | (reportes["Codigo EMP"] == "EMP6877")
    | (reportes["Codigo EMP"] == "EMP6898")
    | (reportes["Codigo EMP"] == "EMP6901")
    | (reportes["Codigo EMP"] == "EMP6905")
    | (reportes["Codigo EMP"] == "EMP7010")
    | (reportes["Codigo EMP"] == "EMP7066")
    | (reportes["Codigo EMP"] == "EMP7080")
    | (reportes["Codigo EMP"] == "EMP7094")
    | (reportes["Codigo EMP"] == "EMP7150")
    | (reportes["Codigo EMP"] == "EMP7333")
    | (reportes["Codigo EMP"] == "EMP7504")
    | (reportes["Codigo EMP"] == "EMP7505")
    | (reportes["Codigo EMP"] == "EMP7623")
    | (reportes["Codigo EMP"] == "EMP7625")
    | (reportes["Codigo EMP"] == "EMP7651")
    | (reportes["Codigo EMP"] == "EMP7721")
    | (reportes["Codigo EMP"] == "EMP7724")
    | (reportes["Corredor"] == "Corredores De Seguros Security Ltda.")
]

Security_metlife.replace({"Clínica": Clinicas_metlife}, inplace=True)
Security_metlife = Security_metlife[
    (Security_metlife["Empleador"] == Security_metlife["Nombre Empresa"])
]
Security_metlife.to_excel("F:/Reportes SF/MetLife/Security_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\3360871456.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Security_metlife.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [68]:
Thb = reportes[
    (reportes["Codigo EMP"] == "EMP9336")
    | (reportes["Codigo EMP"] == "EMP9442")
    | (reportes["Codigo EMP"] == "EMP9746")
    | (reportes["Codigo EMP"] == "EMP9627")
    | (reportes["Codigo EMP"] == "EMP5618")
    | (reportes["Codigo EMP"] == "EMP5619")
    | (reportes["Codigo EMP"] == "EMP5620")
    | (reportes["Codigo EMP"] == "EMP5621")
    | (reportes["Codigo EMP"] == "EMP5622")
    | (reportes["Codigo EMP"] == "EMP5623")
    | (reportes["Codigo EMP"] == "EMP5624")
    | (reportes["Codigo EMP"] == "EMP5625")
    | (reportes["Codigo EMP"] == "EMP5626")
    | (reportes["Codigo EMP"] == "EMP5628")
    | (reportes["Codigo EMP"] == "EMP5629")
    | (reportes["Codigo EMP"] == "EMP5630")
    | (reportes["Codigo EMP"] == "EMP5631")
    | (reportes["Codigo EMP"] == "EMP5632")
    | (reportes["Codigo EMP"] == "EMP5633")
    | (reportes["Codigo EMP"] == "EMP5634")
    | (reportes["Codigo EMP"] == "EMP5635")
    | (reportes["Codigo EMP"] == "EMP5636")
    | (reportes["Codigo EMP"] == "EMP5637")
    | (reportes["Codigo EMP"] == "EMP5638")
    | (reportes["Codigo EMP"] == "EMP5669")
    | (reportes["Codigo EMP"] == "EMP5670")
    | (reportes["Codigo EMP"] == "EMP5671")
    | (reportes["Codigo EMP"] == "EMP5672")
    | (reportes["Codigo EMP"] == "EMP5673")
    | (reportes["Codigo EMP"] == "EMP5674")
    | (reportes["Codigo EMP"] == "EMP5675")
    | (reportes["Codigo EMP"] == "EMP5677")
    | (reportes["Codigo EMP"] == "EMP5721")
    | (reportes["Codigo EMP"] == "EMP5722")
    | (reportes["Codigo EMP"] == "EMP5723")
    | (reportes["Codigo EMP"] == "EMP5724")
    | (reportes["Codigo EMP"] == "EMP5725")
    | (reportes["Codigo EMP"] == "EMP5726")
    | (reportes["Codigo EMP"] == "EMP5727")
    | (reportes["Codigo EMP"] == "EMP5728")
    | (reportes["Codigo EMP"] == "EMP5729")
    | (reportes["Codigo EMP"] == "EMP5730")
    | (reportes["Codigo EMP"] == "EMP5731")
    | (reportes["Codigo EMP"] == "EMP5732")
    | (reportes["Codigo EMP"] == "EMP5733")
    | (reportes["Codigo EMP"] == "EMP5734")
    | (reportes["Codigo EMP"] == "EMP5735")
    | (reportes["Codigo EMP"] == "EMP5736")
    | (reportes["Codigo EMP"] == "EMP5737")
    | (reportes["Codigo EMP"] == "EMP5738")
    | (reportes["Codigo EMP"] == "EMP9242")
    | (reportes["Codigo EMP"] == "EMP9121")
    | (reportes["Codigo EMP"] == "EMP9122")
    | (reportes["Codigo EMP"] == "EMP9123")
    | (reportes["Codigo EMP"] == "EMP9168")
    | (reportes["Codigo EMP"] == "EMP9137")
    | (reportes["Codigo EMP"] == "EMP9320")
    | (reportes["Codigo EMP"] == "EMP9364")
    | (reportes["Codigo EMP"] == "EMP9257")
    | (reportes["Codigo EMP"] == "EMP9258")
    | (reportes["Codigo EMP"] == "EMP9259")
    | (reportes["Codigo EMP"] == "EMP9260")
    | (reportes["Codigo EMP"] == "EMP9261")
    | (reportes["Codigo EMP"] == "EMP9262")
    | (reportes["Codigo EMP"] == "EMP9461")
    | (reportes["Codigo EMP"] == "EMP9463")
    | (reportes["Codigo EMP"] == "EMP9859")
    | (reportes["Codigo EMP"] == "EMP9887")
    | (reportes["Codigo EMP"] == "EMP9888")
    | (reportes["Codigo EMP"] == "EMP9925")
    | (reportes["Codigo EMP"] == "EMP9952")
    | (reportes["Codigo EMP"] == "EMP9953")
    | (reportes["Codigo EMP"] == "EMP9514")
    | (reportes["Codigo EMP"] == "EMP9515")
    | (reportes["Codigo EMP"] == "EMP9516")
    | (reportes["Codigo EMP"] == "EMP9517")
    | (reportes["Codigo EMP"] == "EMP9956")
    | (reportes["Codigo EMP"] == "EMP9957")
    | (reportes["Codigo EMP"] == "EMP9958")
    | (reportes["Codigo EMP"] == "EMP9970")
    | (reportes["Codigo EMP"] == "EMP9559")
    | (reportes["Codigo EMP"] == "EMP9561")
    | (reportes["Codigo EMP"] == "EMP9565")
    | (reportes["Codigo EMP"] == "EMP9567")
    | (reportes["Codigo EMP"] == "EMP9568")
    | (reportes["Codigo EMP"] == "EMP9569")
    | (reportes["Codigo EMP"] == "EMP9570")
    | (reportes["Codigo EMP"] == "EMP9571")
    | (reportes["Codigo EMP"] == "EMP9572")
    | (reportes["Codigo EMP"] == "EMP9573")
    | (reportes["Codigo EMP"] == "EMP9574")
    | (reportes["Codigo EMP"] == "EMP9576")
    | (reportes["Codigo EMP"] == "EMP9698")
    | (reportes["Codigo EMP"] == "EMP9699")
    | (reportes["Codigo EMP"] == "EMP9700")
    | (reportes["Codigo EMP"] == "EMP9577")
    | (reportes["Codigo EMP"] == "EMP9580")
    | (reportes["Codigo EMP"] == "EMP9581")
    | (reportes["Codigo EMP"] == "EMP9583")
    | (reportes["Codigo EMP"] == "EMP9584")
    | (reportes["Codigo EMP"] == "EMP9585")
    | (reportes["Codigo EMP"] == "EMP9586")
    | (reportes["Codigo EMP"] == "EMP9587")
    | (reportes["Codigo EMP"] == "EMP9588")
    | (reportes["Codigo EMP"] == "EMP9594")
    | (reportes["Codigo EMP"] == "EMP9624")
    | (reportes["Codigo EMP"] == "EMP9625")
    | (reportes["Codigo EMP"] == "EMP9626")
    | (reportes["Codigo EMP"] == "EMP9628")
    | (reportes["Codigo EMP"] == "EMP9629")
    | (reportes["Codigo EMP"] == "EMP9633")
    | (reportes["Codigo EMP"] == "EMP9634")
    | (reportes["Codigo EMP"] == "EMP9635")
    | (reportes["Codigo EMP"] == "EMP9645")
    | (reportes["Codigo EMP"] == "EMP9791")
    | (reportes["Codigo EMP"] == "EMP9799")
    | (reportes["Codigo EMP"] == "EMP9806")
    | (reportes["Codigo EMP"] == "EMP9807")
    | (reportes["Codigo EMP"] == "EMP9691")
    | (reportes["Codigo EMP"] == "EMP6373")
    | (reportes["Codigo EMP"] == "EMP6397")
    | (reportes["Codigo EMP"] == "EMP6410")
    | (reportes["Codigo EMP"] == "EMP6481")
    | (reportes["Codigo EMP"] == "EMP6528")
    | (reportes["Codigo EMP"] == "EMP6612")
    | (reportes["Codigo EMP"] == "EMP6613")
    | (reportes["Codigo EMP"] == "EMP6753")
    | (reportes["Codigo EMP"] == "EMP6875")
    | (reportes["Codigo EMP"] == "EMP6891")
    | (reportes["Codigo EMP"] == "EMP7011")
    | (reportes["Codigo EMP"] == "EMP7012")
    | (reportes["Codigo EMP"] == "EMP7039")
    | (reportes["Codigo EMP"] == "EMP7040")
    | (reportes["Codigo EMP"] == "EMP7041")
    | (reportes["Codigo EMP"] == "EMP7051")
    | (reportes["Codigo EMP"] == "EMP7143")
    | (reportes["Codigo EMP"] == "EMP7170")
    | (reportes["Codigo EMP"] == "EMP7339")
    | (reportes["Codigo EMP"] == "EMP7420")
    | (reportes["Codigo EMP"] == "EMP7568")
    | (reportes["Codigo EMP"] == "EMP7626")
    | (reportes["Codigo EMP"] == "EMP7627")
    | (reportes["Codigo EMP"] == "EMP7633")
    | (reportes["Codigo EMP"] == "EMP7701")
    | (reportes["Codigo EMP"] == "EMP7740")
    | (reportes["Codigo EMP"] == "EMP7746")
    | (reportes["Codigo EMP"] == "EMP7772")
    | (reportes["Codigo EMP"] == "EMP7776")
    | (reportes["Corredor"] == "THB Chile Corredores De Seguros S.A.")
]
Thb.replace({"Clínica": Clinicas_metlife}, inplace=True)
Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/MetLife/Thb_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\3869347732.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Thb.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [69]:
Veritas = reportes[
    (reportes["Codigo EMP"] == "EMP5717")
    | (reportes["Codigo EMP"] == "EMP5718")
    | (reportes["Codigo EMP"] == "EMP5719")
    | (reportes["Codigo EMP"] == "EMP5720")
    | (reportes["Codigo EMP"] == "EMP9959")
    | (reportes["Codigo EMP"] == "EMP9707")
    | (reportes["Codigo EMP"] == "EMP9823")
    | (reportes["Codigo EMP"] == "EMP9824")
    | (reportes["Codigo EMP"] == "EMP6615")
    | (reportes["Codigo EMP"] == "EMP6662")
    | (reportes["Codigo EMP"] == "EMP6778")
    | (reportes["Codigo EMP"] == "EMP6872")
    | (reportes["Codigo EMP"] == "EMP7042")
    | (reportes["Codigo EMP"] == "EMP7052")
    | (reportes["Corredor"] == "Veritas Corredores De Seguros S.A.")
]
Veritas.replace({"Clínica": Clinicas_metlife}, inplace=True)
Veritas = Veritas[(Veritas["Empleador"] == Veritas["Nombre Empresa"])]
Veritas.to_excel("F:/Reportes SF/MetLife/Veritas_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2416436682.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Veritas.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [70]:
Mercer = reportes[
    (reportes["Codigo EMP"] == "EMP9162")
    | (reportes["Codigo EMP"] == "EMP5741")
    | (reportes["Codigo EMP"] == "EMP5744")
    | (reportes["Codigo EMP"] == "EMP5755")
    | (reportes["Codigo EMP"] == "EMP5763")
    | (reportes["Codigo EMP"] == "EMP5772")
    | (reportes["Codigo EMP"] == "EMP5795")
    | (reportes["Codigo EMP"] == "EMP5811")
    | (reportes["Codigo EMP"] == "EMP3852")
    | (reportes["Codigo EMP"] == "EMP9377")
    | (reportes["Codigo EMP"] == "EMP9544")
    | (reportes["Codigo EMP"] == "EMP9714")
    | (reportes["Codigo EMP"] == "EMP9715")
    | (reportes["Codigo EMP"] == "EMP9716")
    | (reportes["Codigo EMP"] == "EMP9717")
    | (reportes["Codigo EMP"] == "EMP9721")
    | (reportes["Codigo EMP"] == "EMP9722")
    | (reportes["Codigo EMP"] == "EMP9723")
    | (reportes["Codigo EMP"] == "EMP6464")
    | (reportes["Codigo EMP"] == "EMP7653")
    | (reportes["Codigo EMP"] == "EMP9244")
    | (reportes["Codigo EMP"] == "EMP9245")
    | (reportes["Codigo EMP"] == "EMP9246")
    | (reportes["Codigo EMP"] == "EMP9247")
    | (reportes["Codigo EMP"] == "EMP9248")
    | (reportes["Codigo EMP"] == "EMP9249")
    | (reportes["Codigo EMP"] == "EMP9250")
    | (reportes["Codigo EMP"] == "EMP9140")
    | (reportes["Codigo EMP"] == "EMP9141")
    | (reportes["Codigo EMP"] == "EMP9148")
    | (reportes["Codigo EMP"] == "EMP9149")
    | (reportes["Codigo EMP"] == "EMP9150")
    | (reportes["Codigo EMP"] == "EMP9151")
    | (reportes["Codigo EMP"] == "EMP9152")
    | (reportes["Codigo EMP"] == "EMP9153")
    | (reportes["Codigo EMP"] == "EMP9154")
    | (reportes["Codigo EMP"] == "EMP9155")
    | (reportes["Codigo EMP"] == "EMP9156")
    | (reportes["Codigo EMP"] == "EMP9157")
    | (reportes["Codigo EMP"] == "EMP9158")
    | (reportes["Codigo EMP"] == "EMP9159")
    | (reportes["Codigo EMP"] == "EMP9160")
    | (reportes["Codigo EMP"] == "EMP9161")
    | (reportes["Codigo EMP"] == "EMP9163")
    | (reportes["Codigo EMP"] == "EMP9164")
    | (reportes["Codigo EMP"] == "EMP9165")
    | (reportes["Codigo EMP"] == "EMP9166")
    | (reportes["Codigo EMP"] == "EMP9167")
    | (reportes["Codigo EMP"] == "EMP9127")
    | (reportes["Codigo EMP"] == "EMP9130")
    | (reportes["Codigo EMP"] == "EMP9131")
    | (reportes["Codigo EMP"] == "EMP5742")
    | (reportes["Codigo EMP"] == "EMP5743")
    | (reportes["Codigo EMP"] == "EMP5745")
    | (reportes["Codigo EMP"] == "EMP5746")
    | (reportes["Codigo EMP"] == "EMP5747")
    | (reportes["Codigo EMP"] == "EMP5748")
    | (reportes["Codigo EMP"] == "EMP5749")
    | (reportes["Codigo EMP"] == "EMP5750")
    | (reportes["Codigo EMP"] == "EMP5751")
    | (reportes["Codigo EMP"] == "EMP5752")
    | (reportes["Codigo EMP"] == "EMP5753")
    | (reportes["Codigo EMP"] == "EMP5754")
    | (reportes["Codigo EMP"] == "EMP5756")
    | (reportes["Codigo EMP"] == "EMP5757")
    | (reportes["Codigo EMP"] == "EMP5758")
    | (reportes["Codigo EMP"] == "EMP5759")
    | (reportes["Codigo EMP"] == "EMP5760")
    | (reportes["Codigo EMP"] == "EMP5761")
    | (reportes["Codigo EMP"] == "EMP5762")
    | (reportes["Codigo EMP"] == "EMP5764")
    | (reportes["Codigo EMP"] == "EMP5765")
    | (reportes["Codigo EMP"] == "EMP5766")
    | (reportes["Codigo EMP"] == "EMP5767")
    | (reportes["Codigo EMP"] == "EMP5768")
    | (reportes["Codigo EMP"] == "EMP5769")
    | (reportes["Codigo EMP"] == "EMP5770")
    | (reportes["Codigo EMP"] == "EMP5771")
    | (reportes["Codigo EMP"] == "EMP5773")
    | (reportes["Codigo EMP"] == "EMP5774")
    | (reportes["Codigo EMP"] == "EMP9276")
    | (reportes["Codigo EMP"] == "EMP9362")
    | (reportes["Codigo EMP"] == "EMP5796")
    | (reportes["Codigo EMP"] == "EMP5797")
    | (reportes["Codigo EMP"] == "EMP5798")
    | (reportes["Codigo EMP"] == "EMP5799")
    | (reportes["Codigo EMP"] == "EMP5800")
    | (reportes["Codigo EMP"] == "EMP5801")
    | (reportes["Codigo EMP"] == "EMP5802")
    | (reportes["Codigo EMP"] == "EMP5803")
    | (reportes["Codigo EMP"] == "EMP5804")
    | (reportes["Codigo EMP"] == "EMP5805")
    | (reportes["Codigo EMP"] == "EMP5806")
    | (reportes["Codigo EMP"] == "EMP5807")
    | (reportes["Codigo EMP"] == "EMP5808")
    | (reportes["Codigo EMP"] == "EMP5809")
    | (reportes["Codigo EMP"] == "EMP5810")
    | (reportes["Codigo EMP"] == "EMP5812")
    | (reportes["Codigo EMP"] == "EMP9264")
    | (reportes["Codigo EMP"] == "EMP9265")
    | (reportes["Codigo EMP"] == "EMP9266")
    | (reportes["Codigo EMP"] == "EMP9267")
    | (reportes["Codigo EMP"] == "EMP9268")
    | (reportes["Codigo EMP"] == "EMP9269")
    | (reportes["Codigo EMP"] == "EMP9270")
    | (reportes["Codigo EMP"] == "EMP9271")
    | (reportes["Codigo EMP"] == "EMP9273")
    | (reportes["Codigo EMP"] == "EMP9274")
    | (reportes["Codigo EMP"] == "EMP9275")
    | (reportes["Codigo EMP"] == "EMP9263")
    | (reportes["Codigo EMP"] == "EMP6178")
    | (reportes["Codigo EMP"] == "EMP6179")
    | (reportes["Codigo EMP"] == "EMP6180")
    | (reportes["Codigo EMP"] == "EMP9445")
    | (reportes["Codigo EMP"] == "EMP9446")
    | (reportes["Codigo EMP"] == "EMP9448")
    | (reportes["Codigo EMP"] == "EMP9449")
    | (reportes["Codigo EMP"] == "EMP9450")
    | (reportes["Codigo EMP"] == "EMP9451")
    | (reportes["Codigo EMP"] == "EMP9462")
    | (reportes["Codigo EMP"] == "EMP9480")
    | (reportes["Codigo EMP"] == "EMP9867")
    | (reportes["Codigo EMP"] == "EMP9868")
    | (reportes["Codigo EMP"] == "EMP9869")
    | (reportes["Codigo EMP"] == "EMP9932")
    | (reportes["Codigo EMP"] == "EMP9933")
    | (reportes["Codigo EMP"] == "EMP9943")
    | (reportes["Codigo EMP"] == "EMP9944")
    | (reportes["Codigo EMP"] == "EMP9945")
    | (reportes["Codigo EMP"] == "EMP9964")
    | (reportes["Codigo EMP"] == "EMP9965")
    | (reportes["Codigo EMP"] == "EMP9596")
    | (reportes["Codigo EMP"] == "EMP9713")
    | (reportes["Codigo EMP"] == "EMP9724")
    | (reportes["Codigo EMP"] == "EMP9725")
    | (reportes["Codigo EMP"] == "EMP9726")
    | (reportes["Codigo EMP"] == "EMP9727")
    | (reportes["Codigo EMP"] == "EMP9728")
    | (reportes["Codigo EMP"] == "EMP9729")
    | (reportes["Codigo EMP"] == "EMP9730")
    | (reportes["Codigo EMP"] == "EMP9731")
    | (reportes["Codigo EMP"] == "EMP9732")
    | (reportes["Codigo EMP"] == "EMP9733")
    | (reportes["Codigo EMP"] == "EMP9738")
    | (reportes["Codigo EMP"] == "EMP9636")
    | (reportes["Codigo EMP"] == "EMP9640")
    | (reportes["Codigo EMP"] == "EMP9641")
    | (reportes["Codigo EMP"] == "EMP9642")
    | (reportes["Codigo EMP"] == "EMP9643")
    | (reportes["Codigo EMP"] == "EMP9644")
    | (reportes["Codigo EMP"] == "EMP6366")
    | (reportes["Codigo EMP"] == "EMP6425")
    | (reportes["Codigo EMP"] == "EMP6433")
    | (reportes["Codigo EMP"] == "EMP6445")
    | (reportes["Codigo EMP"] == "EMP6446")
    | (reportes["Codigo EMP"] == "EMP6450")
    | (reportes["Codigo EMP"] == "EMP6547")
    | (reportes["Codigo EMP"] == "EMP6625")
    | (reportes["Codigo EMP"] == "EMP6626")
    | (reportes["Codigo EMP"] == "EMP6648")
    | (reportes["Codigo EMP"] == "EMP6655")
    | (reportes["Codigo EMP"] == "EMP6656")
    | (reportes["Codigo EMP"] == "EMP6657")
    | (reportes["Codigo EMP"] == "EMP6658")
    | (reportes["Codigo EMP"] == "EMP6659")
    | (reportes["Codigo EMP"] == "EMP6780")
    | (reportes["Codigo EMP"] == "EMP6791")
    | (reportes["Codigo EMP"] == "EMP6897")
    | (reportes["Codigo EMP"] == "EMP6931")
    | (reportes["Codigo EMP"] == "EMP6941")
    | (reportes["Codigo EMP"] == "EMP6942")
    | (reportes["Codigo EMP"] == "EMP6968")
    | (reportes["Codigo EMP"] == "EMP6970")
    | (reportes["Codigo EMP"] == "EMP7074")
    | (reportes["Codigo EMP"] == "EMP7325")
    | (reportes["Codigo EMP"] == "EMP7342")
    | (reportes["Codigo EMP"] == "EMP7415")
    | (reportes["Codigo EMP"] == "EMP7425")
    | (reportes["Codigo EMP"] == "EMP7527")
    | (reportes["Codigo EMP"] == "EMP7529")
    | (reportes["Codigo EMP"] == "EMP7622")
    | (reportes["Codigo EMP"] == "EMP7628")
    | (reportes["Codigo EMP"] == "EMP7644")
    | (reportes["Codigo EMP"] == "EMP7645")
    | (reportes["Codigo EMP"] == "EMP7710")
    | (reportes["Codigo EMP"] == "EMP7711")
    | (reportes["Codigo EMP"] == "EMP7773")
    | (reportes["Codigo EMP"] == "EMP7774")
    | (reportes["Codigo EMP"] == "EMP7775")
    | (reportes["Codigo EMP"] == "EMP7790")
    | (reportes["Codigo EMP"] == "EMP7792")
    | (reportes["Codigo EMP"] == "EMP7874")
    | (reportes["Codigo EMP"] == "EMP7879")
    | (reportes["Corredor"] == "Mercer Corredores De Seguros Ltda.")
]
Mercer.replace({"Clínica": Clinicas_metlife}, inplace=True)
Mercer = Mercer[(Mercer["Empleador"] == Mercer["Nombre Empresa"])]
Mercer.to_excel("F:/Reportes SF/MetLife/Mercer_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\1490380631.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mercer.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [71]:
Mercado_Libre = reportes[(reportes["Codigo EMP"] == "EMP5617")]

Mercado_Libre = Mercado_Libre[
    (Mercado_Libre["Empleador"] == Mercado_Libre["Nombre Empresa"])
]
Mercado_Libre.to_excel(
    "F:/Reportes SF/Mercado Libre/Mercado_Libre_Base.xlsx", index=False
)

In [72]:
Ahumada = reportes[(reportes["Codigo EMP"] == "EMP6475")]

Ahumada = Ahumada[(Ahumada["Empleador"] == Ahumada["Nombre Empresa"])]
Ahumada.to_excel("F:/Reportes SF/Farmacias Ahumada/Ahumada_Base.xlsx", index=False)

In [73]:
Minera_Abra = reportes[
    (reportes["Codigo EMP"] == "EMP7397")
    | (reportes["Codigo EMP"] == "EMP7795")
    | (reportes["Codigo EMP"] == "EMP7796")
]

Minera_Abra = Minera_Abra[(Minera_Abra["Empleador"] == Minera_Abra["Nombre Empresa"])]
Minera_Abra.to_excel("F:/Reportes SF/Minera El Abra/Abra_Base.xlsx", index=False)

In [74]:
Agrosuper = reportes[(reportes["Holding"] == "Agrosuper")]
Agrosuper.replace({"Clínica": Clinicas_metlife}, inplace=True)
Agrosuper = Agrosuper[(Agrosuper["Empleador"] == Agrosuper["Nombre Empresa"])]
Agrosuper.to_excel("F:/Reportes SF/Metlife/Mercer_Agrosuper_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2211481513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Agrosuper.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [75]:
Agrosuper2 = reportes[(reportes["Holding"] == "Agrosuper 2")]
Agrosuper2.replace({"Clínica": Clinicas_metlife}, inplace=True)
# Agrosuper2 = Agrosuper2[(Agrosuper2["Empleador"] == Agrosuper2["Nombre Empresa"])]
Agrosuper2.to_excel("F:/Reportes SF/Metlife/Mercer_Agrosuper2_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\989549224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Agrosuper2.replace({"Clínica": Clinicas_metlife}, inplace=True)


In [76]:
Lipigas = reportes[(reportes["Codigo EMP"] == "EMP7310")]

Lipigas = Lipigas[(Lipigas["Empleador"] == Lipigas["Nombre Empresa"])]
Lipigas.to_excel("F:/Reportes SF/Lipigas/Lipigas_Base.xlsx", index=False)

In [77]:
Socofar = reportes[
    (reportes["Codigo EMP"] == "EMP3780")
    | (reportes["Codigo EMP"] == "EMP6749")
    | (reportes["Codigo EMP"] == "EMP6750")
    | (reportes["Codigo EMP"] == "EMP6751")
    | (reportes["Codigo EMP"] == "EMP7405")
    | (reportes["Codigo EMP"] == "EMP7406")
    | (reportes["Codigo EMP"] == "EMP7407")
]


Socofar = Socofar[(Socofar["Empleador"] == Socofar["Nombre Empresa"])]
Socofar.to_excel("F:/Reportes SF/Socofar/Socofar_Base.xlsx", index=False)

Socofar1 = Socofar[(Socofar["Mes"] == 1)]
Socofar1.to_excel("F:/Reportes SF/Socofar/Socofar_Enero_Base.xlsx", index=False)

Socofar2 = Socofar[(Socofar["Mes"] == 2)]
Socofar2.to_excel("F:/Reportes SF/Socofar/Socofar_Febrero_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 3)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Marzo_Base.xlsx", index=False)


In [78]:
Kaufmann = reportes[
    (reportes["Codigo EMP"] == "EMP4171")
    | (reportes["Codigo EMP"] == "EMP7308")
    | (reportes["Codigo EMP"] == "EMP7309")
    | (reportes["Campaña"] == "Kaufmann")
]


Kaufmann = Kaufmann[(Kaufmann["Empleador"] == Kaufmann["Nombre Empresa"])]
Kaufmann.to_excel("F:/Reportes SF/Kaufmann/Kaufmann_Base.xlsx", index=False)


In [79]:
Tressp = reportes[
    (reportes["Codigo EMP"] == "EMP6467")
    # | (reportes["Codigo EMP"] == "EMP6468")
    # | (reportes["Codigo EMP"] == "EMP7607")
]


Tressp = Tressp[(Tressp["Empleador"] == Tressp["Nombre Empresa"])]
Tressp.to_excel("F:/Reportes SF/3SP/3sp_Base.xlsx", index=False)
Tressp.shape

(8, 27)

BCI


In [80]:
Bci = reportes[(reportes["Campaña"] == "BCI")]


Bci = Bci[
    (Bci["Empleador"] == Bci["Nombre Empresa"])
    & (Bci["Codigo EMP"] != "EMP6254")
    & (Bci["Codigo EMP"] != "EMP6255")
    & (reportes["Holding"] != "Orbis")
]
Bci.to_excel("F:/Reportes SF/BCI/BCI_Base.xlsx", index=False)

Cenin = reportes[
    (reportes["Codigo EMP"] == "EMP6250") | (reportes["Holding"] == "Cemin")
]
Cenin = Cenin[(Cenin["Empleador"] == Cenin["Nombre Empresa"])]
Cenin.to_excel("F:/Reportes SF/BCI/Cenin_Base.xlsx", index=False)

Orbis = reportes[
    (reportes["Codigo EMP"] == "EMP6254")
    | (reportes["Codigo EMP"] == "EMP6255")
    | (reportes["Holding"] == "Orbis")
]
Orbis = Orbis[(Orbis["Empleador"] == Orbis["Nombre Empresa"])]
Orbis.to_excel("F:/Reportes SF/BCI/Orbis_Base.xlsx", index=False)

Embonor = reportes[(reportes["Holding"] == "Embonor")]
Embonor = Embonor[(Embonor["Empleador"] == Embonor["Nombre Empresa"])]
Embonor.to_excel("F:/Reportes SF/BCI/Embonor_Base.xlsx", index=False)


Saesa = reportes[(reportes["Holding"] == "Saesa")]
Saesa = Saesa[(Saesa["Empleador"] == Saesa["Nombre Empresa"])]
Saesa.to_excel("F:/Reportes SF/BCI/Saesa_Base.xlsx", index=False)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\2279787881.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Bci = Bci[


In [81]:
Colmena = reportes[(reportes["Campaña"] == "Colmena")]
clinicas_colmena = {"Mis Primeros Pasos": "Deporte"}
Colmena.replace({"Clínica": clinicas_colmena}, inplace=True)


Colmena = Colmena[(Colmena["Empleador"] == Colmena["Nombre Empresa"])]
Colmena.to_excel("F:/Reportes SF/Colmena/Colmena_Base.xlsx", index=False)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12052\955374149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Colmena.replace({"Clínica": clinicas_colmena}, inplace=True)


In [82]:
Pedro = reportes[(reportes["Campaña"] == "Pedro De Valdivia")]


Pedro = Pedro[(Pedro["Empleador"] == Pedro["Nombre Empresa"])]
Pedro.to_excel("F:/Reportes SF/Pedro de Valdivia/Pedro_Base.xlsx", index=False)

In [83]:
Vidacel = reportes[(reportes["Campaña"] == "VidaCel")]


Vidacel = Vidacel[(Vidacel["Empleador"] == Vidacel["Nombre Empresa"])]
Vidacel.to_excel("F:/Reportes SF/Vidacel/Vidacel_Base.xlsx", index=False)

VidacelColab = reportes[(reportes["Campaña"] == "VidaCel Colaboradores")]

VidacelColab = VidacelColab[
    (VidacelColab["Empleador"] == VidacelColab["Nombre Empresa"])
]
VidacelColab.to_excel("F:/Reportes SF/Vidacel/VidacelColab_Base.xlsx", index=False)

In [84]:
Wfs = reportes[(reportes["Campaña"] == "Holding Wfs Food Services")]

Wfs = Wfs[(Wfs["Empleador"] == Wfs["Nombre Empresa"])]
Wfs.to_excel("F:/Reportes SF/Wfs/Wfs_Base.xlsx", index=False)

In [85]:
Dermic = reportes[(reportes["Campaña"] == "Dermik")]

Dermic = Dermic[(Dermic["Empleador"] == Dermic["Nombre Empresa"])]
Dermic.to_excel("F:/Reportes SF/Dermik/Dermic_Base.xlsx", index=False)
Dermic.shape

(198, 27)

In [86]:
Instruvalve = reportes[(reportes["Campaña"] == "Instruvalve")]

Instruvalve = Instruvalve[(Instruvalve["Empleador"] == Instruvalve["Nombre Empresa"])]
Instruvalve.to_excel("F:/Reportes SF/Instruvalve/Instruvalve_Base.xlsx", index=False)

In [87]:
Thb = reportes[(reportes["Campaña"] == "THB Seguros")]

Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/THB Seguros/Thb_Base.xlsx", index=False)

In [88]:
Construmart = reportes[(reportes["Campaña"] == "Construmart")]
Construmart = Construmart[(Construmart["Fecha"] > "2023-02-01")]

Clinicas_Construmart = {
    "Pacientes Crónicos": "Telemedicina",
    "Clínica Del Sueño": "Telemedicina",
}
Construmart.replace({"Clínica": Clinicas_Construmart}, inplace=True)
Construmart = Construmart[(Construmart["Empleador"] == Construmart["Nombre Empresa"])]
Construmart["Agendamiento"] = "No"
# np.where(Construmart['Codigo Ficha'].str.contains('FSA') == True, 'Si', Construmart['agendamiento'])
Construmart["Agendamiento"] = np.where(
    Construmart["Codigo Ficha"].str.contains("FSA") == True,
    "Si",
    Construmart["Agendamiento"],
)
# Construmart['Codigo Ficha'].star
# Construmart["agendamiento"][Construmart['Codigo Ficha'].str.contains('FSA')]

Construmart.to_excel("F:/Reportes SF/Construmart/Construmart_Base.xlsx", index=False)

In [89]:
reportesMX = reportes[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Campaña",
        "Empleador",
        "Fecha",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Vigente",
        "Pais",
    ]
]
Mexico = reportesMX[(reportesMX["Pais"] == "México")]
Mexico = Mexico.rename(columns={"Rut": "CURP"})

Mexico


In [90]:


Mexico = Mexico[(Mexico["Empleador"] == Mexico["Nombre Empresa"])]
Mexico.to_excel("F:/Reportes SF/Mexico/Mexico_Base.xlsx", index=False)

In [91]:
CareU = Mexico[(Mexico["Campaña"] == "Care U")]

CareU = CareU[(CareU["Empleador"] == CareU["Nombre Empresa"])]
CareU.to_excel("F:/Reportes SF/Mexico/Care U/Care_Base.xlsx", index=False)

In [92]:
Bbdo = Mexico[(Mexico["Campaña"] == "BBDO")]

Bbdo = Bbdo[(Bbdo["Empleador"] == Bbdo["Nombre Empresa"])]
Bbdo.to_excel("F:/Reportes SF/Mexico/BBDO/BBDO_Base.xlsx", index=False)

In [93]:
Riddell = Mexico[(Mexico["Campaña"] == "Riddell Sports")]

Riddell = Riddell[(Riddell["Empleador"] == Riddell["Nombre Empresa"])]
Riddell.to_excel(
    "F:/Reportes SF/Mexico/Riddell Sports/Riddell Sports_Base.xlsx", index=False
)

In [94]:
Cuartel = Mexico[(Mexico["Campaña"] == "Cuartel General")]

Cuartel = Cuartel[(Cuartel["Empleador"] == Cuartel["Nombre Empresa"])]
Cuartel.to_excel("F:/Reportes SF/Mexico/Cuartel General/Cuartel_Base.xlsx", index=False)

In [95]:
Ducter = Mexico[(Mexico["Campaña"] == "Ducter")]

Ducter = Ducter[(Ducter["Empleador"] == Ducter["Nombre Empresa"])]
Ducter.to_excel("F:/Reportes SF/Mexico/Ducter/Ducter_Base.xlsx", index=False)

In [96]:
Efectivale = Mexico[(Mexico["Campaña"] == "Efectivale")]

Efectivale = Efectivale[(Efectivale["Empleador"] == Efectivale["Nombre Empresa"])]
Efectivale.to_excel(
    "F:/Reportes SF/Mexico/Efectivale/Efectivale_Base.xlsx", index=False
)

In [97]:
Ambar = Mexico[(Mexico["Campaña"] == "Ambar")]

Ambar = Ambar[(Ambar["Empleador"] == Ambar["Nombre Empresa"])]
Ambar.to_excel("F:/Reportes SF/Mexico/Ambar/Ambar_Base.xlsx", index=False)

In [98]:
Charger = Mexico[(Mexico["Campaña"] == "Charger Logistics")]

Charger = Charger[(Charger["Empleador"] == Charger["Nombre Empresa"])]
Charger.to_excel(
    "F:/Reportes SF/Mexico/Charger Logistics/Charger Logistics_Base.xlsx", index=False
)

In [99]:
Clinica = Mexico[(Mexico["Campaña"] == "Clínica Virtual")]

Clinica = Clinica[(Clinica["Empleador"] == Clinica["Nombre Empresa"])]
Clinica.to_excel(
    "F:/Reportes SF/Mexico/Clinica Virtual/Clinica Virtual_Base.xlsx", index=False
)

In [100]:
Omg = Mexico[(Mexico["Campaña"] == "OMG")]

Omg = Omg[(Omg["Empleador"] == Omg["Nombre Empresa"])]
Omg.to_excel("F:/Reportes SF/Mexico/OMG/OMG_Base.xlsx", index=False)

In [101]:
Efisys = Mexico[(Mexico["Campaña"] == "Efisys")]

Efisys = Efisys[(Efisys["Empleador"] == Efisys["Nombre Empresa"])]
Efisys.to_excel("F:/Reportes SF/Mexico/Efisys/Efisys_Base.xlsx", index=False)

In [102]:
Fadermex = Mexico[(Mexico["Campaña"] == "Fadermex")]

Fadermex = Fadermex[(Fadermex["Empleador"] == Fadermex["Nombre Empresa"])]
Fadermex.to_excel("F:/Reportes SF/Mexico/Fadermex/Fadermex_Base.xlsx", index=False)

In [103]:
Farmacias = Mexico[(Mexico["Campaña"] == "Farmacias YZA")]

Farmacias = Farmacias[(Farmacias["Empleador"] == Farmacias["Nombre Empresa"])]
Farmacias.to_excel(
    "F:/Reportes SF/Mexico/Farmacias YZA/Farmacias YZA_Base.xlsx", index=False
)

In [104]:
Geodis = Mexico[(Mexico["Campaña"] == "Geodis")]

Geodis = Geodis[(Geodis["Empleador"] == Geodis["Nombre Empresa"])]
Geodis.to_excel("F:/Reportes SF/Mexico/Geodis/Geodis_Base.xlsx", index=False)

In [105]:
Hidrosina = Mexico[(Mexico["Campaña"] == "Hidrosina")]

Hidrosina = Hidrosina[(Hidrosina["Empleador"] == Hidrosina["Nombre Empresa"])]
Hidrosina.to_excel("F:/Reportes SF/Mexico/Hidrosina/Hidrosina_Base.xlsx", index=False)

In [106]:
Icsi = Mexico[(Mexico["Campaña"] == "ICSI")]

Icsi = Icsi[(Icsi["Empleador"] == Icsi["Nombre Empresa"])]
Icsi.to_excel("F:/Reportes SF/Mexico/ICSI/ICSI_Base.xlsx", index=False)

In [107]:
Infra = Mexico[(Mexico["Campaña"] == "Infra")]

Infra = Infra[(Infra["Empleador"] == Infra["Nombre Empresa"])]
Infra.to_excel("F:/Reportes SF/Mexico/Infra/Infra_Base.xlsx", index=False)



aldo = Mexico[(Mexico["Campaña"] == "Aldo Conti")]

aldo = aldo[(aldo["Empleador"] == aldo["Nombre Empresa"])]
aldo.to_excel("F:/Reportes SF/Mexico/Aldo Conti/Aldo_Base.xlsx", index=False)


In [108]:
Cursor = Mexico[(Mexico["Campaña"] == "Mi Cursor")]

Cursor = Cursor[(Cursor["Empleador"] == Cursor["Nombre Empresa"])]
Cursor.to_excel("F:/Reportes SF/Mexico/Mi Cursor/Mi Cursor_Base.xlsx", index=False)

In [109]:
Porter = Mexico[(Mexico["Campaña"] == "Porter Novelli")]

Porter = Porter[(Porter["Empleador"] == Porter["Nombre Empresa"])]
Porter.to_excel(
    "F:/Reportes SF/Mexico/Porter Novelli/Porter Novelli_Base.xlsx", index=False
)

In [110]:
Productos = Mexico[(Mexico["Campaña"] == "Productos San Juan")]

Productos = Productos[(Productos["Empleador"] == Productos["Nombre Empresa"])]
Productos.to_excel(
    "F:/Reportes SF/Mexico/Productos San Juan/Productos San Juan_Base.xlsx", index=False
)

In [111]:
Reckitt = Mexico[(Mexico["Campaña"] == "Reckitt Benckiser")]

Reckitt = Reckitt[(Reckitt["Empleador"] == Reckitt["Nombre Empresa"])]
Reckitt.to_excel(
    "F:/Reportes SF/Mexico/Reckitt Benckiser/Reckitt Benckiser_Base.xlsx", index=False
)

In [112]:
Sintec = Mexico[(Mexico["Campaña"] == "Sintec")]

Sintec = Sintec[(Sintec["Empleador"] == Sintec["Nombre Empresa"])]
Sintec.to_excel("F:/Reportes SF/Mexico/Sintec/Sintec_Base.xlsx", index=False)

In [113]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Metlife.to_excel("C:/Users/jmartinez/Documents/Metlife.xlsx", index=False)

Security = reportes[
    (reportes["Campaña"] == "Vida Security")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Security.to_excel("C:/Users/jmartinez/Documents/Vida Security.xlsx", index=False)

# Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)
reportes_francisca = Reportes_con_empresas
reportes_francisca.Empleador_x.fillna("Nones", inplace=True)
reportes_francisca = reportes_francisca[reportes_francisca["Empleador_x"] == "Nones"]
archivo = "SinEmpresas.xlsx"
archivo1 = "Empresas.xlsx"

fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
# archivo_con_fecha = archivo_con_fecha.replace(".", "_")
hasta = Path(
    "E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Pacientes Sin Empresas/"
).joinpath(archivo_con_fecha)
print(hasta)
reportes_francisca.to_excel(hasta, index=False)

E:\Siempre\Usuarios\Francisca Rubio Yáñez\TI\Pacientes Sin Empresas\2023_07_03_SinEmpresas.xlsx


In [114]:
fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo1}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Empresas/").joinpath(
    archivo_con_fecha
)
Programas_pacientes.to_excel(hasta, index=False)

hasta = Path("F:/Empresas SF/empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(hasta, index=False)